In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import files
import os, shutil

uploaded = files.upload()
os.makedirs('/content/drive/MyDrive/swissmetro', exist_ok=True)

for fn in uploaded.keys():
    shutil.move(fn, f'/content/drive/MyDrive/swissmetro/{fn}')

print("Saved to Drive:", os.listdir('/content/drive/MyDrive/swissmetro')[:5])


Saving swissmetro.dat to swissmetro.dat
Saved to Drive: ['artifacts', 'swissmetro.dat']


In [4]:
import pandas as pd

DATA_PATH = '/content/drive/MyDrive/swissmetro/swissmetro.dat'
df = pd.read_csv(DATA_PATH, sep='\t')

print("rows:", len(df))
print("respondents:", df['ID'].nunique())
print("choices per person:", df.groupby('ID').size().unique())
print(df[['GROUP','ID','CHOICE']].head())

rows: 10728
respondents: 1192
choices per person: [9]
   GROUP  ID  CHOICE
0      2   1       2
1      2   1       2
2      2   1       2
3      2   1       2
4      2   1       2


In [5]:
import pandas as pd

DATA_PATH = '/content/drive/MyDrive/swissmetro/swissmetro.dat'

df = pd.read_csv(DATA_PATH, sep=r"\s+", engine="python")

print("rows:", len(df))
print("respondents:", df["ID"].nunique())
print("choices per person:", df.groupby("ID").size().unique())
df.head()

rows: 10728
respondents: 1192
choices per person: [9]


,GROUP,SURVEY,SP,ID,PURPOSE,FIRST,TICKET,WHO,LUGGAGE,AGE,...,TRAIN_TT,TRAIN_CO,TRAIN_HE,SM_TT,SM_CO,SM_HE,SM_SEATS,CAR_TT,CAR_CO,CHOICE
0,2,0,1,1,1,0,1,1,0,3,...,112,48,120,63,52,20,0,117,65,2
1,2,0,1,1,1,0,1,1,0,3,...,103,48,30,60,49,10,0,117,84,2
2,2,0,1,1,1,0,1,1,0,3,...,130,48,60,67,58,30,0,117,52,2
3,2,0,1,1,1,0,1,1,0,3,...,103,40,30,63,52,20,0,72,52,2
4,2,0,1,1,1,0,1,1,0,3,...,130,36,60,63,42,20,0,90,84,2


In [6]:
import numpy as np
from sklearn.model_selection import train_test_split

df = df.loc[df["CHOICE"] > 0].copy()

ids = df["ID"].unique()
train_ids, val_ids = train_test_split(ids, test_size=0.2, random_state=42)

df_train = df[df["ID"].isin(train_ids)].copy()
df_val   = df[df["ID"].isin(val_ids)].copy()

print("train respondents:", df_train["ID"].nunique(), "rows:", len(df_train))
print("val respondents:",   df_val["ID"].nunique(),   "rows:", len(df_val))

train respondents: 952 rows: 8568
val respondents: 239 rows: 2151


In [7]:
def build_matrices(df_in):
    df2 = df_in.copy()
    N = len(df2)

    # y: 1=Train,2=SM,3=Car -> 0/1/2
    y = df2["CHOICE"].to_numpy(int) - 1

    # availability
    AV = np.column_stack([
        df2["TRAIN_AV"].to_numpy(float),
        df2["SM_AV"].to_numpy(float),
        df2["CAR_AV"].to_numpy(float),
    ])

    # time (scaled for numeric stability)
    TT = np.column_stack([
        df2["TRAIN_TT"].to_numpy(float),
        df2["SM_TT"].to_numpy(float),
        df2["CAR_TT"].to_numpy(float),
    ]) / 100.0

    # cost: 方案B（Train边际票价；SM_CO原样）
    GA = df2["GA"].to_numpy(int)
    TRAIN_FARE = df2["TRAIN_CO"].to_numpy(float) * (GA == 0)  # GA=1 -> 0
    SM_COST    = df2["SM_CO"].to_numpy(float)                 # 不考虑GA（保持原样）
    CAR_COST   = df2["CAR_CO"].to_numpy(float)

    CO = np.column_stack([TRAIN_FARE, SM_COST, CAR_COST]) / 100.0

    # headway: Car无headway -> 0
    HE = np.column_stack([
        df2["TRAIN_HE"].to_numpy(float),
        df2["SM_HE"].to_numpy(float),
        np.zeros(N),
    ]) / 100.0

    # seats: only for SM
    SM_SEATS = df2["SM_SEATS"].to_numpy(float)
    SEATS = np.column_stack([
        np.zeros(N),
        SM_SEATS,
        np.zeros(N),
    ])

    # individual vars (raw)
    ID = df2["ID"].to_numpy(int)
    FIRST  = df2["FIRST"].to_numpy(float)
    MALE   = df2["MALE"].to_numpy(float)
    AGE    = df2["AGE"].to_numpy(int)       # 1..6
    INCOME = df2["INCOME"].to_numpy(int)    # 0..4
    PURPOSE= df2["PURPOSE"].to_numpy(int)   # 1..9

    return {
        "N": N, "y": y, "AV": AV, "TT": TT, "CO": CO, "HE": HE, "SEATS": SEATS,
        "ID": ID, "FIRST": FIRST, "MALE": MALE, "AGE": AGE, "INCOME": INCOME, "PURPOSE": PURPOSE
    }

train = build_matrices(df_train)
val   = build_matrices(df_val)

(train["N"], val["N"])


(8568, 2151)

In [8]:
# ===== Cell 2.5: 只用 train 计算缩放参数，然后同时缩放 train/val =====
import numpy as np

# 只在 train 上计算每列的标准差（按备选项列：Train/SM/Car）
tt_scale = train["TT"].std(axis=0, keepdims=True) + 1e-12
co_scale = train["CO"].std(axis=0, keepdims=True) + 1e-12
he_scale = train["HE"].std(axis=0, keepdims=True) + 1e-12

# 生成缩放后的版本（不覆盖原始TT/CO/HE，方便你回溯）
train["TTs"] = train["TT"] / tt_scale
val["TTs"]   = val["TT"]   / tt_scale

train["COs"] = train["CO"] / co_scale
val["COs"]   = val["CO"]   / co_scale

train["HEs"] = train["HE"] / he_scale
val["HEs"]   = val["HE"]   / he_scale


In [9]:
def onehot(x, levels, drop_first=True):
    x = np.asarray(x)
    mats = [(x == lv).astype(float) for lv in levels]
    M = np.column_stack(mats)
    return M[:, 1:] if drop_first else M

def softmax_with_av(U, AV):
    # U: (N,3), AV: (N,3)
    U_masked = np.where(AV > 0, U, -1e10)
    Umax = np.max(U_masked, axis=1, keepdims=True)
    expU = np.exp(U_masked - Umax)
    denom = np.sum(expU, axis=1, keepdims=True)
    P = expU / denom
    return P

def neg_loglike_from_P(P, y):
    p = P[np.arange(len(y)), y]
    return -np.sum(np.log(p + 1e-300))

def accuracy(P, y):
    pred = np.argmax(P, axis=1)
    return (pred == y).mean()

In [10]:
from scipy.optimize import minimize

def fit_mnl_step1(data):
    N, y, AV = data["N"], data["y"], data["AV"]
    TT, CO, HE, SEATS = data["TTs"], data["COs"], data["HEs"], data["SEATS"]

    # theta = [b_tt, b_co, b_he, b_seats, asc_sm, asc_car]
    def neg_ll(theta):
        b_tt, b_co, b_he, b_seats, asc_sm, asc_car = theta
        U = b_tt*TT + b_co*CO + b_he*HE + b_seats*SEATS
        U[:,1] += asc_sm
        U[:,2] += asc_car
        P = softmax_with_av(U, AV)
        return neg_loglike_from_P(P, y)

    x0 = np.zeros(6)
    res = minimize(neg_ll, x0, method="L-BFGS-B")
    return res

res1 = fit_mnl_step1(train)
print("Converged:", res1.success, res1.message)
print("negLL(train):", res1.fun)

theta1 = res1.x
names1 = ["B_TT","B_CO","B_HE","B_SEATS","ASC_SM","ASC_CAR"]
for n,v in zip(names1, theta1):
    print(f"{n:10s}: {v: .6f}")


Converged: True CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
negLL(train): 7063.978480231855
B_TT      : -0.808699
B_CO      : -0.232134
B_HE      : -0.113127
B_SEATS   :  0.251140
ASC_SM    :  1.007408
ASC_CAR   :  0.521936


In [11]:
def predict_step1(theta, data):
    TT, CO, HE, SEATS = data["TTs"], data["COs"], data["HEs"], data["SEATS"]
    AV = data["AV"]
    b_tt, b_co, b_he, b_seats, asc_sm, asc_car = theta

    U = b_tt*TT + b_co*CO + b_he*HE + b_seats*SEATS
    U[:,1] += asc_sm
    U[:,2] += asc_car
    return softmax_with_av(U, AV)

P_tr_1 = predict_step1(theta1, train)
P_va_1 = predict_step1(theta1, val)

print("Step1 train LL:", -neg_loglike_from_P(P_tr_1, train["y"]), "acc:", accuracy(P_tr_1, train["y"]))
print("Step1 val   LL:", -neg_loglike_from_P(P_va_1, val["y"]),   "acc:", accuracy(P_va_1, val["y"]))


Step1 train LL: -7063.978480231855 acc: 0.6045751633986928
Step1 val   LL: -1741.1731212964232 acc: 0.604834960483496


In [12]:
def build_X_ind(data):
    FIRST = data["FIRST"]
    MALE  = data["MALE"]
    AGE   = data["AGE"]
    INCOME= data["INCOME"]
    PURPOSE = data["PURPOSE"]

    X_age    = onehot(AGE,    levels=[1,2,3,4,5,6], drop_first=True)     # baseline: 1
    X_income = onehot(INCOME, levels=[0,1,2,3,4],   drop_first=True)     # baseline: 0
    X_purp   = onehot(PURPOSE,levels=[1,2,3,4,5,6,7,8,9], drop_first=True) # baseline: 1

    X = np.column_stack([FIRST, MALE, X_age, X_income, X_purp])
    return X

Xtr = build_X_ind(train)
Xva = build_X_ind(val)

print("X_ind shape:", Xtr.shape)


X_ind shape: (8568, 19)


In [13]:
from scipy.optimize import minimize
import numpy as np

def fit_mnl_step2(data, X_ind, theta1=None):
    N, y, AV = data["N"], data["y"], data["AV"]
    TT, CO, HE, SEATS = data["TTs"], data["COs"], data["HEs"], data["SEATS"]

    K = X_ind.shape[1]

    def neg_ll(theta):
        b_tt, b_co, b_he, b_seats, asc_sm, asc_car = theta[:6]
        gamma_sm  = theta[6:6+K]
        gamma_car = theta[6+K:6+2*K]

        U = b_tt*TT + b_co*CO + b_he*HE + b_seats*SEATS
        U[:,1] += asc_sm + X_ind @ gamma_sm
        U[:,2] += asc_car + X_ind @ gamma_car

        P = softmax_with_av(U, AV)
        return neg_loglike_from_P(P, y)

    if theta1 is None:
        x0 = np.zeros(6 + 2*K)
    else:
        x0 = np.concatenate([theta1, np.zeros(2*K)])

    res = minimize(
        neg_ll, x0, method="L-BFGS-B",
        options={"maxiter": 5000, "maxfun": 5000}
    )
    return res

def predict_step2(theta, data, X_ind):
    TT, CO, HE, SEATS, AV = data["TTs"], data["COs"], data["HEs"], data["SEATS"], data["AV"]
    K = X_ind.shape[1]

    b_tt, b_co, b_he, b_seats, asc_sm, asc_car = theta[:6]
    gamma_sm  = theta[6:6+K]
    gamma_car = theta[6+K:6+2*K]

    U = b_tt*TT + b_co*CO + b_he*HE + b_seats*SEATS
    U[:,1] += asc_sm + X_ind @ gamma_sm
    U[:,2] += asc_car + X_ind @ gamma_car

    return softmax_with_av(U, AV)

res2 = fit_mnl_step2(train, Xtr, theta1=theta1)
print("Converged:", res2.success, res2.message)
print("negLL(train):", res2.fun)

theta2 = res2.x

P_tr_2 = predict_step2(theta2, train, Xtr)
P_va_2 = predict_step2(theta2, val,   Xva)

print("Step2 train LL:", -neg_loglike_from_P(P_tr_2, train["y"]), "acc:", accuracy(P_tr_2, train["y"]))
print("Step2 val   LL:", -neg_loglike_from_P(P_va_2, val["y"]),   "acc:", accuracy(P_va_2, val["y"]))

print("Δ val LL (Step2 - Step1):",
      (-neg_loglike_from_P(P_va_2, val["y"])) - (-neg_loglike_from_P(P_va_1, val["y"])))


Converged: False STOP: TOTAL NO. OF F,G EVALUATIONS EXCEEDS LIMIT
negLL(train): 6447.051911043493
Step2 train LL: -6447.051911043493 acc: 0.6537114845938375
Step2 val   LL: -1635.3721016618986 acc: 0.6415620641562064
Δ val LL (Step2 - Step1): 105.80101963452466


In [14]:
import numpy as np, os

SAVE_DIR = "/content/drive/MyDrive/swissmetro/artifacts"
os.makedirs(SAVE_DIR, exist_ok=True)

np.save(f"{SAVE_DIR}/theta_step1.npy", theta1)
np.save(f"{SAVE_DIR}/theta_step2.npy", theta2)

print("Saved to:", SAVE_DIR)


Saved to: /content/drive/MyDrive/swissmetro/artifacts


In [15]:
import numpy as np
from scipy.optimize import minimize

# 1) 构造一个外部可调用的 negLL（闭包：把 train/Xtr 固定住）
def make_negll_step2(data, X_ind):
    y, AV = data["y"], data["AV"]
    TT, CO, HE, SEATS = data["TTs"], data["COs"], data["HEs"], data["SEATS"]
    K = X_ind.shape[1]

    def negll(theta):
        b_tt, b_co, b_he, b_seats, asc_sm, asc_car = theta[:6]
        gamma_sm  = theta[6:6+K]
        gamma_car = theta[6+K:6+2*K]

        U = b_tt*TT + b_co*CO + b_he*HE + b_seats*SEATS
        U[:,1] += asc_sm + X_ind @ gamma_sm
        U[:,2] += asc_car + X_ind @ gamma_car

        P = softmax_with_av(U, AV)
        return neg_loglike_from_P(P, y)

    return negll

# 2) 续跑函数
def continue_lbfgsb(negll, x0, rounds=3, maxfun=20000, maxiter=20000, ftol=1e-6, gtol=1e-5):
    x = x0
    res = None
    for i in range(rounds):
        res = minimize(
            negll, x, method="L-BFGS-B",
            options={"maxfun": maxfun, "maxiter": maxiter, "ftol": ftol, "gtol": gtol}
        )
        print(f"[round {i+1}] success={res.success}  nfev={res.nfev}  njev={res.njev}  fun={res.fun}")
        x = res.x
        if res.success:
            break
    return res

# 3) 真正执行：用你刚才 step2 的结果 res2.x 当初值续跑
negll_train = make_negll_step2(train, Xtr)
res2_cont = continue_lbfgsb(negll_train, res2.x, rounds=3)

theta2 = res2_cont.x
print("final success:", res2_cont.success, res2_cont.message)
print("final negLL(train):", res2_cont.fun)


[round 1] success=True  nfev=2115  njev=47  fun=6445.6847827330685
final success: True CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
final negLL(train): 6445.6847827330685


In [16]:
P_tr_2 = predict_step2(theta2, train, Xtr)
P_va_2 = predict_step2(theta2, val,   Xva)

print("Step2 train LL:", -neg_loglike_from_P(P_tr_2, train["y"]), "acc:", accuracy(P_tr_2, train["y"]))
print("Step2 val   LL:", -neg_loglike_from_P(P_va_2, val["y"]),   "acc:", accuracy(P_va_2, val["y"]))

print("Δ val LL (Step2 - Step1):",
      (-neg_loglike_from_P(P_va_2, val["y"])) - (-neg_loglike_from_P(P_va_1, val["y"])))


Step2 train LL: -6445.6847827330685 acc: 0.6546451914098973
Step2 val   LL: -1634.4977837374702 acc: 0.6438865643886564
Δ val LL (Step2 - Step1): 106.67533755895306


In [17]:
import numpy as np
import pandas as pd

def get_X_names():
    names = []
    # 你 build_X_ind 的顺序：FIRST, MALE, AGE(2..6), INCOME(1..4), PURPOSE(2..9)
    names += ["FIRST"]
    names += ["MALE"]
    names += [f"AGE_{k}" for k in [2,3,4,5,6]]          # baseline AGE=1
    names += [f"INCOME_{k}" for k in [1,2,3,4]]         # baseline INCOME=0
    names += [f"PURPOSE_{k}" for k in [2,3,4,5,6,7,8,9]]# baseline PURPOSE=1
    return names

def unpack_theta2(theta, K):
    head = theta[:6]
    gamma_sm  = theta[6:6+K]
    gamma_car = theta[6+K:6+2*K]
    return head, gamma_sm, gamma_car

# ===== 1) 打印 step2 的头部系数（和 step1 对齐）=====
names_head = ["B_TT","B_CO","B_HE","B_SEATS","ASC_SM","ASC_CAR"]
print("Step2 head params:")
for n,v in zip(names_head, theta2[:6]):
    print(f"{n:10s}: {v: .6f}")

# ===== 2) 打印个体变量的 gamma（SM/Car）=====
X_names = get_X_names()
K = Xtr.shape[1]
assert K == len(X_names), (K, len(X_names))

_, gamma_sm, gamma_car = unpack_theta2(theta2, K)

df_gamma = pd.DataFrame({
    "feature": X_names,
    "gamma_SM": gamma_sm,
    "gamma_CAR": gamma_car,
})
df_gamma["abs_max"] = np.maximum(np.abs(df_gamma["gamma_SM"]), np.abs(df_gamma["gamma_CAR"]))
df_gamma = df_gamma.sort_values("abs_max", ascending=False).reset_index(drop=True)

display(df_gamma.head(15))   # 看影响最大的前15个


Step2 head params:
B_TT      : -0.709402
B_CO      : -0.252284
B_HE      : -0.122764
B_SEATS   :  0.088525
ASC_SM    :  1.667612
ASC_CAR   : -0.012332


,feature,gamma_SM,gamma_CAR,abs_max
0,PURPOSE_5,-0.205297,-6.052574,6.052574
1,PURPOSE_8,-2.649589,-3.510649,3.510649
2,INCOME_4,-1.442716,-1.899939,1.899939
3,PURPOSE_7,-0.622093,-1.556304,1.556304
4,PURPOSE_4,0.236664,1.516115,1.516115
5,INCOME_1,-1.331066,-1.305282,1.331066
6,AGE_5,-1.070946,-0.244701,1.070946
7,AGE_2,0.704330,0.967651,0.967651
8,AGE_3,0.614102,0.958290,0.958290
9,PURPOSE_2,-0.548249,-0.936211,0.936211


In [18]:
from scipy.optimize import minimize

def fit_step2_given_cols(data, X_ind_sub, theta1=None, maxfun=20000, maxiter=20000, ftol=1e-6, gtol=1e-5):
    y, AV = data["y"], data["AV"]
    TT, CO, HE, SEATS = data["TTs"], data["COs"], data["HEs"], data["SEATS"]  # ✅一定用缩放后的
    K = X_ind_sub.shape[1]

    def negll(theta):
        b_tt, b_co, b_he, b_seats, asc_sm, asc_car = theta[:6]
        gamma_sm  = theta[6:6+K]
        gamma_car = theta[6+K:6+2*K]

        U = b_tt*TT + b_co*CO + b_he*HE + b_seats*SEATS
        U[:,1] += asc_sm + X_ind_sub @ gamma_sm
        U[:,2] += asc_car + X_ind_sub @ gamma_car

        P = softmax_with_av(U, AV)
        return neg_loglike_from_P(P, y)

    if theta1 is None:
        x0 = np.zeros(6 + 2*K)
    else:
        x0 = np.concatenate([theta1, np.zeros(2*K)])

    res = minimize(
        negll, x0, method="L-BFGS-B",
        options={"maxfun": maxfun, "maxiter": maxiter, "ftol": ftol, "gtol": gtol}
    )
    return res

def predict_step2_sub(theta, data, X_ind_sub):
    TT, CO, HE, SEATS, AV = data["TTs"], data["COs"], data["HEs"], data["SEATS"], data["AV"]
    K = X_ind_sub.shape[1]

    b_tt, b_co, b_he, b_seats, asc_sm, asc_car = theta[:6]
    gamma_sm  = theta[6:6+K]
    gamma_car = theta[6+K:6+2*K]

    U = b_tt*TT + b_co*CO + b_he*HE + b_seats*SEATS
    U[:,1] += asc_sm + X_ind_sub @ gamma_sm
    U[:,2] += asc_car + X_ind_sub @ gamma_car
    return softmax_with_av(U, AV)

# ===== 0) 准备列名与分组 =====
X_names = get_X_names()
name_to_idx = {n:i for i,n in enumerate(X_names)}

def cols_by_prefix(prefix):
    return [i for i,n in enumerate(X_names) if n.startswith(prefix)]

idx_FIRST   = [name_to_idx["FIRST"]]
idx_MALE    = [name_to_idx["MALE"]]
idx_AGE     = cols_by_prefix("AGE_")
idx_INCOME  = cols_by_prefix("INCOME_")
idx_PURPOSE = cols_by_prefix("PURPOSE_")

# ===== 1) 定义“逐步加入”的方案（你可以改顺序）=====
steps = [
    ("MALE + INCOME",                idx_MALE + idx_INCOME),
    ("+ AGE",                        idx_MALE + idx_INCOME + idx_AGE),
    ("+ PURPOSE",                    idx_MALE + idx_INCOME + idx_AGE + idx_PURPOSE),
    ("+ FIRST (full X_ind)",         idx_MALE + idx_INCOME + idx_AGE + idx_PURPOSE + idx_FIRST),
]

# ===== 2) 先拿 Step1 的基线表现 =====
base_val_LL  = -neg_loglike_from_P(P_va_1, val["y"])
base_val_acc = accuracy(P_va_1, val["y"])

rows = []
rows.append({
    "model": "Step1 (context only)",
    "K": 0,
    "val_LL": base_val_LL,
    "val_acc": base_val_acc,
    "Δval_LL_vs_Step1": 0.0,
    "converged": True
})

# ===== 3) 逐步拟合 Step2 子模型 =====
for label, idxs in steps:
    Xtr_sub = Xtr[:, idxs]
    Xva_sub = Xva[:, idxs]

    res = fit_step2_given_cols(train, Xtr_sub, theta1=theta1)
    theta = res.x

    Pva = predict_step2_sub(theta, val, Xva_sub)
    val_LL  = -neg_loglike_from_P(Pva, val["y"])
    val_acc = accuracy(Pva, val["y"])

    rows.append({
        "model": f"Step2 {label}",
        "K": len(idxs),
        "val_LL": val_LL,
        "val_acc": val_acc,
        "Δval_LL_vs_Step1": val_LL - base_val_LL,
        "converged": bool(res.success),
    })
    print(label, "| converged:", res.success, "| val_LL:", val_LL, "| val_acc:", val_acc)

df_ablation = pd.DataFrame(rows)
display(df_ablation)


MALE + INCOME | converged: True | val_LL: -1707.5059548596134 | val_acc: 0.6118084611808461
+ AGE | converged: True | val_LL: -1680.799275223485 | val_acc: 0.6127382612738261
+ PURPOSE | converged: True | val_LL: -1634.839073883987 | val_acc: 0.6438865643886564
+ FIRST (full X_ind) | converged: True | val_LL: -1634.2961242856798 | val_acc: 0.6457461645746164


,model,K,val_LL,val_acc,Δval_LL_vs_Step1,converged
0,Step1 (context only),0,-1741.173121,0.604835,0.000000,True
1,Step2 MALE + INCOME,5,-1707.505955,0.611808,33.667166,True
2,Step2 + AGE,10,-1680.799275,0.612738,60.373846,True
3,Step2 + PURPOSE,18,-1634.839074,0.643887,106.334047,True
4,Step2 + FIRST (full X_ind),19,-1634.296124,0.645746,106.876997,True


In [19]:
import numpy as np
from scipy.optimize import minimize

def fit_null_asc(data):
    y, AV = data["y"], data["AV"]
    N = data["N"]

    # theta = [asc_sm, asc_car]
    def negll(theta):
        asc_sm, asc_car = theta
        U = np.zeros((N, 3))
        U[:,1] = asc_sm
        U[:,2] = asc_car
        P = softmax_with_av(U, AV)
        return neg_loglike_from_P(P, y)

    res = minimize(negll, np.zeros(2), method="L-BFGS-B", options={"maxiter": 5000, "maxfun": 5000})
    return res

res_null = fit_null_asc(train)
theta_null = res_null.x
print("Null converged:", res_null.success, res_null.message)
print("Null negLL(train):", res_null.fun)
print("Null ASC_SM, ASC_CAR:", theta_null)


Null converged: True CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
Null negLL(train): 7579.615254664614
Null ASC_SM, ASC_CAR: [1.48048158 1.0083829 ]


In [20]:
# 复用你之前的 get_X_names / name_to_idx / cols_by_prefix
X_names = get_X_names()
name_to_idx = {n:i for i,n in enumerate(X_names)}

def cols_by_prefix(prefix):
    return [i for i,n in enumerate(X_names) if n.startswith(prefix)]

idx_MALE    = [name_to_idx["MALE"]]
idx_AGE     = cols_by_prefix("AGE_")
idx_INCOME  = cols_by_prefix("INCOME_")
idx_PURPOSE = cols_by_prefix("PURPOSE_")

idx_step2_main = idx_MALE + idx_INCOME + idx_AGE + idx_PURPOSE  # ✅ 不含 FIRST

Xtr_main = Xtr[:, idx_step2_main]
Xva_main = Xva[:, idx_step2_main]

# 重新拟合（用 theta1 当初值，收敛更稳）
res2_main = fit_step2_given_cols(train, Xtr_main, theta1=theta1)
print("Step2(main) converged:", res2_main.success, res2_main.message)

theta2_main = res2_main.x
P_tr_2m = predict_step2_sub(theta2_main, train, Xtr_main)
P_va_2m = predict_step2_sub(theta2_main, val,   Xva_main)

LL_tr_1 = -neg_loglike_from_P(P_tr_1, train["y"])
LL_va_1 = -neg_loglike_from_P(P_va_1, val["y"])
LL_tr_2m = -neg_loglike_from_P(P_tr_2m, train["y"])
LL_va_2m = -neg_loglike_from_P(P_va_2m, val["y"])

acc_tr_1 = accuracy(P_tr_1, train["y"])
acc_va_1 = accuracy(P_va_1, val["y"])
acc_tr_2m = accuracy(P_tr_2m, train["y"])
acc_va_2m = accuracy(P_va_2m, val["y"])

print("Step1 val LL / acc:", LL_va_1, acc_va_1)
print("Step2(main) val LL / acc:", LL_va_2m, acc_va_2m)
print("Δ val LL:", LL_va_2m - LL_va_1)


Step2(main) converged: True CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
Step1 val LL / acc: -1741.1731212964232 0.604834960483496
Step2(main) val LL / acc: -1634.839073883987 0.6438865643886564
Δ val LL: 106.33404741243612


In [21]:
import pandas as pd

LL0_tr = -res_null.fun  # Null 的 train LL（负数）
# 注意：res_null.fun 是 negLL，所以 LL0_tr = -negLL

rho2_step1 = 1 - (LL_tr_1 / LL0_tr)
rho2_step2 = 1 - (LL_tr_2m / LL0_tr)

df_summary = pd.DataFrame([
    {
        "Model": "Null (ASC only)",
        "Train LL": LL0_tr,
        "Val LL": np.nan,
        "Train acc": np.nan,
        "Val acc": np.nan,
        "Δ Val LL vs Step1": np.nan,
        "McFadden rho^2 (train)": np.nan
    },
    {
        "Model": "Step1 (context only)",
        "Train LL": LL_tr_1,
        "Val LL": LL_va_1,
        "Train acc": acc_tr_1,
        "Val acc": acc_va_1,
        "Δ Val LL vs Step1": 0.0,
        "McFadden rho^2 (train)": rho2_step1
    },
    {
        "Model": "Step2 main (context + MALE/INCOME/AGE/PURPOSE)",
        "Train LL": LL_tr_2m,
        "Val LL": LL_va_2m,
        "Train acc": acc_tr_2m,
        "Val acc": acc_va_2m,
        "Δ Val LL vs Step1": LL_va_2m - LL_va_1,
        "McFadden rho^2 (train)": rho2_step2
    }
])

display(df_summary)


,Model,Train LL,Val LL,Train acc,Val acc,Δ Val LL vs Step1,McFadden rho^2 (train)
0,Null (ASC only),-7579.615255,NaN,NaN,NaN,NaN,NaN
1,Step1 (context only),-7063.978480,-1741.173121,0.604575,0.604835,0.000000,0.068029
2,Step2 main (context + MALE/INCOME/AGE/PURPOSE),-6457.928077,-1634.839074,0.651027,0.643887,106.334047,0.147987


In [22]:
import numpy as np
import pandas as pd
from scipy.stats import norm

# ===== 1) 从 L-BFGS-B 拿到近似的 inv(Hessian) =====
def get_hess_inv_dense(res):
    Hinv = res.hess_inv
    # L-BFGS-B 的 hess_inv 可能是 LinearOperator
    if hasattr(Hinv, "todense"):
        Hinv = np.asarray(Hinv.todense())
    else:
        # 有些版本需要这样把 LinearOperator 变成矩阵
        Hinv = np.asarray(Hinv(np.eye(len(res.x))))
    return Hinv

Hinv = get_hess_inv_dense(res2_main)

# ===== 2) 计算 SE / z / p =====
theta = res2_main.x
diag = np.diag(Hinv)

# 数值近似下可能出现非常小的负数（数值误差），clip 一下避免 sqrt 报错
diag = np.clip(diag, 0, None)
se = np.sqrt(diag)

z = theta / se
p = 2 * (1 - norm.cdf(np.abs(z)))

def stars(p):
    if p < 0.001: return "***"
    if p < 0.01:  return "**"
    if p < 0.05:  return "*"
    if p < 0.1:   return "."
    return ""

# ===== 3) 生成参数名字（与你的 Xtr_main 列顺序严格对齐）=====
head_names = ["B_TT","B_CO","B_HE","B_SEATS","ASC_SM","ASC_CAR"]

# Xtr_main 的列顺序来自 idx_step2_main（你已经构造好了）
feat_names_main = [X_names[i] for i in idx_step2_main]  # 例如 MALE, INCOME_1.., AGE_2.., PURPOSE_2..

K = len(feat_names_main)
assert len(theta) == 6 + 2*K, (len(theta), 6+2*K)

gamma_sm_names  = [f"G_SM:{n}"  for n in feat_names_main]
gamma_car_names = [f"G_CAR:{n}" for n in feat_names_main]

param_names = head_names + gamma_sm_names + gamma_car_names

# ===== 4) 输出回归表 =====
df_coef = pd.DataFrame({
    "param": param_names,
    "coef": theta,
    "se": se,
    "z": z,
    "p": p,
})
df_coef["sig"] = df_coef["p"].apply(stars)

# （可选）按 |z| 排序看最显著的变量
df_top = df_coef.sort_values("z", key=lambda s: np.abs(s), ascending=False).reset_index(drop=True)

print("Step2(main) converged:", res2_main.success, res2_main.message)
display(df_coef.head(10))     # 前10个（头部参数 + 部分gamma）
display(df_top.head(20))      # 最显著的20个参数


Step2(main) converged: True CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH


,param,coef,se,z,p,sig
0,B_TT,-0.708775,1.057876,-0.669998,0.502859,
1,B_CO,-0.251370,0.319823,-0.785966,0.431887,
2,B_HE,-0.123570,0.586124,-0.210826,0.833023,
3,B_SEATS,0.080875,1.040064,0.077760,0.938019,
4,ASC_SM,1.800950,1.374874,1.309902,0.190229,
5,ASC_CAR,0.250457,1.378262,0.181719,0.855803,
6,G_SM:MALE,0.495903,1.464715,0.338566,0.734937,
7,G_SM:INCOME_1,-1.466684,3.047855,-0.481218,0.630361,
8,G_SM:INCOME_2,-0.989174,1.181260,-0.837389,0.402374,
9,G_SM:INCOME_3,-0.646967,1.174090,-0.551037,0.581608,


,param,coef,se,z,p,sig
0,G_SM:INCOME_4,-1.540472,1.101711,-1.398255,0.162037,
1,ASC_SM,1.800950,1.374874,1.309902,0.190229,
2,G_CAR:PURPOSE_4,1.439684,1.195998,1.203751,0.228686,
3,G_CAR:INCOME_4,-2.039333,1.932960,-1.055031,0.291411,
4,G_SM:AGE_5,-1.015073,0.972757,-1.043501,0.296716,
5,G_SM:INCOME_2,-0.989174,1.181260,-0.837389,0.402374,
6,G_CAR:PURPOSE_7,-1.610855,1.944417,-0.828451,0.407415,
7,G_CAR:PURPOSE_5,-6.233236,7.697265,-0.809799,0.418056,
8,G_CAR:INCOME_1,-1.532307,1.948438,-0.786428,0.431617,
9,B_CO,-0.251370,0.319823,-0.785966,0.431887,


In [23]:
cols = ["TRAIN_TT","SM_TT","CAR_TT","TRAIN_CO","SM_CO","CAR_CO","TRAIN_HE","SM_HE"]
print(df_train[cols].describe().loc[["mean","std","min","max"]])

         TRAIN_TT       SM_TT       CAR_TT     TRAIN_CO        SM_CO  \
mean   166.253385   88.356559   123.945261   498.395775   650.189192   
std     79.415476   56.476958    89.940716  1065.223848  1413.172851   
min     31.000000    8.000000     0.000000     9.000000    11.000000   
max   1049.000000  796.000000  1560.000000  5040.000000  6720.000000   

          CAR_CO    TRAIN_HE      SM_HE  
mean   79.050654   70.164566  19.990663  
std    55.777983   37.434769   8.161147  
min     0.000000   30.000000  10.000000  
max   520.000000  120.000000  30.000000  


In [24]:
import numpy as np
import pandas as pd

# ---------- onehot & X_ind（与你现在的19维一致） ----------
def onehot(x, levels, drop_first=True):
    x = np.asarray(x).astype(int)
    levels = list(levels)
    cols = levels[1:] if drop_first else levels
    out = np.column_stack([(x == lv).astype(float) for lv in cols]) if len(cols) > 0 else np.zeros((len(x), 0))
    return out

def get_X_names():
    # 19 维：FIRST, MALE, AGE_2..6(5), INCOME_1..4(4), PURPOSE_2..9(8)
    names = ["FIRST", "MALE"]
    names += [f"AGE_{k}" for k in [2,3,4,5,6]]
    names += [f"INCOME_{k}" for k in [1,2,3,4]]
    names += [f"PURPOSE_{k}" for k in [2,3,4,5,6,7,8,9]]
    return names

X_names = get_X_names()
name_to_idx = {n:i for i,n in enumerate(X_names)}

# 主模型 Step2(main)：不含 FIRST
idx_MALE    = [name_to_idx["MALE"]]
idx_AGE     = [name_to_idx[n] for n in X_names if n.startswith("AGE_")]
idx_INCOME  = [name_to_idx[n] for n in X_names if n.startswith("INCOME_")]
idx_PURPOSE = [name_to_idx[n] for n in X_names if n.startswith("PURPOSE_")]
idx_step2_main = idx_MALE + idx_INCOME + idx_AGE + idx_PURPOSE  # ✅ 不含 FIRST

def build_X_ind_from_df(df):
    FIRST   = df["FIRST"].to_numpy(dtype=float)
    MALE    = df["MALE"].to_numpy(dtype=float)
    AGE     = df["AGE"].to_numpy(dtype=int)
    INCOME  = df["INCOME"].to_numpy(dtype=int)
    PURPOSE = df["PURPOSE"].to_numpy(dtype=int)

    X_age    = onehot(AGE,    levels=[1,2,3,4,5,6], drop_first=True)     # baseline AGE=1
    X_income = onehot(INCOME, levels=[0,1,2,3,4],   drop_first=True)     # baseline INCOME=0
    X_purp   = onehot(PURPOSE,levels=[1,2,3,4,5,6,7,8,9], drop_first=True) # baseline PURPOSE=1

    X = np.column_stack([FIRST, MALE, X_age, X_income, X_purp])
    return X

# ---------- 从 df 构建 data dict（含缩放后的 TTs/Cos/HEs） ----------
def build_data_from_df(df, tt_scale, co_scale, he_scale):
    # y: 0/1/2
    y = df["CHOICE"].to_numpy(dtype=int) - 1
    N = len(df)

    # availability
    if all(c in df.columns for c in ["TRAIN_AV", "SM_AV", "CAR_AV"]):
        AV = np.column_stack([
            df["TRAIN_AV"].to_numpy(dtype=float),
            df["SM_AV"].to_numpy(dtype=float),
            df["CAR_AV"].to_numpy(dtype=float),
        ])
    else:
        AV = np.ones((N, 3), dtype=float)

    # TT
    TT = np.column_stack([
        df["TRAIN_TT"].to_numpy(dtype=float),
        df["SM_TT"].to_numpy(dtype=float),
        df["CAR_TT"].to_numpy(dtype=float),
    ])

    # CO (方案B：Train 边际成本，GA=1 -> 0；SM 不考虑 GA)
    GA = df["GA"].to_numpy(dtype=int) if "GA" in df.columns else np.zeros(N, dtype=int)

    CO_train = df["TRAIN_CO"].to_numpy(dtype=float) * (GA == 0)   # ✅ 方案B
    CO_sm    = df["SM_CO"].to_numpy(dtype=float)                  # ✅ 文档：without considering GA
    CO_car   = df["CAR_CO"].to_numpy(dtype=float)

    CO = np.column_stack([CO_train, CO_sm, CO_car])

    # HE：Train/SM 有，Car 设 0
    HE_train = df["TRAIN_HE"].to_numpy(dtype=float) if "TRAIN_HE" in df.columns else np.zeros(N)
    HE_sm    = df["SM_HE"].to_numpy(dtype=float)    if "SM_HE" in df.columns else np.zeros(N)
    HE_car   = np.zeros(N, dtype=float)

    HE = np.column_stack([HE_train, HE_sm, HE_car])

    # SEATS：只对 SM 有（Train/Car = 0）
    seats_col = "SM_SEATS" if "SM_SEATS" in df.columns else ("SEATS" if "SEATS" in df.columns else None)
    if seats_col is None:
        SEATS_sm = np.zeros(N, dtype=float)
    else:
        SEATS_sm = df[seats_col].to_numpy(dtype=float)

    SEATS = np.column_stack([np.zeros(N), SEATS_sm, np.zeros(N)]).astype(float)

    # 缩放（用 train 统计量，广播到(1,3)）
    TTs = TT / tt_scale
    Cos = CO / co_scale
    HEs = HE / he_scale

    return {
        "N": N,
        "y": y,
        "AV": AV,
        "TTs": TTs,
        "Cos": Cos,
        "HEs": HEs,
        "SEATS": SEATS,
    }

# ---------- 你要的核心：给任意 df_sub -> (data_sub, X_sub_main) ----------
def make_data_X_from_df(df_sub, tt_scale, co_scale, he_scale):
    df_sub = df_sub.loc[df_sub["CHOICE"] > 0].copy()  # ✅ 保持一致：过滤 unknown choice
    data_sub = build_data_from_df(df_sub, tt_scale, co_scale, he_scale)
    X_full = build_X_ind_from_df(df_sub)
    X_sub = X_full[:, idx_step2_main]                 # ✅ main: 不含 FIRST
    return data_sub, X_sub

# ---------- 一次性在 train 上计算缩放参数（后面 bootstrap 全复用） ----------
def compute_scales_from_train(df_train):
    df_train = df_train.loc[df_train["CHOICE"] > 0].copy()
    GA = df_train["GA"].to_numpy(dtype=int) if "GA" in df_train.columns else np.zeros(len(df_train), dtype=int)

    TT = np.column_stack([
        df_train["TRAIN_TT"].to_numpy(dtype=float),
        df_train["SM_TT"].to_numpy(dtype=float),
        df_train["CAR_TT"].to_numpy(dtype=float),
    ])

    CO = np.column_stack([
        df_train["TRAIN_CO"].to_numpy(dtype=float) * (GA == 0),
        df_train["SM_CO"].to_numpy(dtype=float),
        df_train["CAR_CO"].to_numpy(dtype=float),
    ])

    HE_train = df_train["TRAIN_HE"].to_numpy(dtype=float) if "TRAIN_HE" in df_train.columns else np.zeros(len(df_train))
    HE_sm    = df_train["SM_HE"].to_numpy(dtype=float)    if "SM_HE" in df_train.columns else np.zeros(len(df_train))
    HE = np.column_stack([HE_train, HE_sm, np.zeros(len(df_train))])

    tt_scale = TT.std(axis=0, keepdims=True) + 1e-12
    co_scale = CO.std(axis=0, keepdims=True) + 1e-12
    he_scale = HE.std(axis=0, keepdims=True) + 1e-12
    return tt_scale, co_scale, he_scale

# 你只要保证 df_train_raw 已经存在（你现在肯定有）
tt_scale, co_scale, he_scale = compute_scales_from_train(df_train)

print("tt_scale:", tt_scale)
print("co_scale:", co_scale)
print("he_scale:", he_scale)
print("K_main:", len(idx_step2_main))


tt_scale: [[79.41084186 56.47366179 89.93546766]]
co_scale: [[  68.26926263 1413.09038034   55.77472782]]
he_scale: [[3.74325848e+01 8.16067102e+00 1.00000000e-12]]
K_main: 18


In [25]:
# Cell 2: sanity check (train/val)

data_tr, Xtr = make_data_X_from_df(df_train, tt_scale, co_scale, he_scale)
data_va, Xva = make_data_X_from_df(df_val, tt_scale, co_scale, he_scale)

print("N_train:", data_tr["N"], "Xtr:", Xtr.shape)
print("N_val  :", data_va["N"], "Xva:", Xva.shape)

# y 分布（0=Train,1=SM,2=Car）
ytr = data_tr["y"]
yva = data_va["y"]
print("y_train counts (0,1,2):", np.bincount(ytr, minlength=3))
print("y_val   counts (0,1,2):", np.bincount(yva, minlength=3))

# availability 是否合理
print("AV_train sum by alt:", data_tr["AV"].sum(axis=0))
print("AV_val   sum by alt:", data_va["AV"].sum(axis=0))

# 是否存在“全部不可用”的行（理论上不应该）
print("any all-unavailable in train?:", np.any(data_tr["AV"].sum(axis=1) == 0))
print("any all-unavailable in val?  :", np.any(data_va["AV"].sum(axis=1) == 0))

N_train: 8568 Xtr: (8568, 18)
N_val  : 2151 Xva: (2151, 18)
y_train counts (0,1,2): [1134 4984 2450]
y_val   counts (0,1,2): [ 289 1232  630]
AV_train sum by alt: [8568. 8568. 7272.]
AV_val   sum by alt: [2151. 2151. 1764.]
any all-unavailable in train?: False
any all-unavailable in val?  : False


In [26]:
from scipy.optimize import minimize
import numpy as np

def fit_step2_main(df_sub, tt_scale, co_scale, he_scale, theta_init=None, maxiter=20000, maxfun=20000):
    data, X = make_data_X_from_df(df_sub, tt_scale, co_scale, he_scale)

    y, AV = data["y"], data["AV"]
    TT, CO, HE, SEATS = data["TTs"], data["Cos"], data["HEs"], data["SEATS"]
    K = X.shape[1]  # should be 18

    def neg_ll(theta):
        b_tt, b_co, b_he, b_seats, asc_sm, asc_car = theta[:6]
        gamma_sm  = theta[6:6+K]
        gamma_car = theta[6+K:6+2*K]

        U = b_tt*TT + b_co*CO + b_he*HE + b_seats*SEATS
        U[:, 1] += asc_sm  + X @ gamma_sm
        U[:, 2] += asc_car + X @ gamma_car

        P = softmax_with_av(U, AV)
        return neg_loglike_from_P(P, y)

    if theta_init is None:
        theta_init = np.zeros(6 + 2*K)

    res = minimize(
        neg_ll, theta_init,
        method="L-BFGS-B",
        options={"maxiter": maxiter, "maxfun": maxfun}
    )
    return res


In [27]:
res2 = fit_step2_main(df_train, tt_scale, co_scale, he_scale)

print("Step2(main) converged:", res2.success, res2.message)
print("negLL(train):", res2.fun)

theta2_main = res2.x
print("theta2_main dim:", len(theta2_main))


Step2(main) converged: False STOP: TOTAL NO. OF F,G EVALUATIONS EXCEEDS LIMIT
negLL(train): 6457.117020724935
theta2_main dim: 42


In [28]:
def predict_step2_main(theta, df_sub, tt_scale, co_scale, he_scale):
    data, X = make_data_X_from_df(df_sub, tt_scale, co_scale, he_scale)

    y, AV = data["y"], data["AV"]
    TT, CO, HE, SEATS = data["TTs"], data["Cos"], data["HEs"], data["SEATS"]
    K = X.shape[1]

    b_tt, b_co, b_he, b_seats, asc_sm, asc_car = theta[:6]
    gamma_sm  = theta[6:6+K]
    gamma_car = theta[6+K:6+2*K]

    U = b_tt*TT + b_co*CO + b_he*HE + b_seats*SEATS
    U[:, 1] += asc_sm  + X @ gamma_sm
    U[:, 2] += asc_car + X @ gamma_car

    P = softmax_with_av(U, AV)
    return P, y

P_tr, y_tr = predict_step2_main(theta2_main, df_train, tt_scale, co_scale, he_scale)
P_va, y_va = predict_step2_main(theta2_main, df_val,   tt_scale, co_scale, he_scale)

LL_tr = -neg_loglike_from_P(P_tr, y_tr)
LL_va = -neg_loglike_from_P(P_va, y_va)

print("Step2(main) train LL:", LL_tr, "acc:", accuracy(P_tr, y_tr))
print("Step2(main) val   LL:", LL_va, "acc:", accuracy(P_va, y_va))
print("Δ val LL vs Step1:", (LL_va - (-neg_loglike_from_P(P_va, y_va))) )  # 这行其实恒为0，可忽略


Step2(main) train LL: -6457.117020724935 acc: 0.6510270774976658
Step2(main) val   LL: -1635.6282220648823 acc: 0.6462110646211064
Δ val LL vs Step1: 0.0


In [29]:
def predict_step2_main(theta, df_sub, tt_scale, co_scale, he_scale):
    data, X = make_data_X_from_df(df_sub, tt_scale, co_scale, he_scale)

    y, AV = data["y"], data["AV"]
    TT, CO, HE, SEATS = data["TTs"], data["Cos"], data["HEs"], data["SEATS"]
    K = X.shape[1]

    b_tt, b_co, b_he, b_seats, asc_sm, asc_car = theta[:6]
    gamma_sm  = theta[6:6+K]
    gamma_car = theta[6+K:6+2*K]

    U = b_tt*TT + b_co*CO + b_he*HE + b_seats*SEATS
    U[:, 1] += asc_sm  + X @ gamma_sm
    U[:, 2] += asc_car + X @ gamma_car

    P = softmax_with_av(U, AV)
    return P, y

P_tr, y_tr = predict_step2_main(theta2_main, df_train, tt_scale, co_scale, he_scale)
P_va, y_va = predict_step2_main(theta2_main, df_val,   tt_scale, co_scale, he_scale)

LL_tr = -neg_loglike_from_P(P_tr, y_tr)
LL_va = -neg_loglike_from_P(P_va, y_va)

print("Step2(main) train LL:", LL_tr, "acc:", accuracy(P_tr, y_tr))
print("Step2(main) val   LL:", LL_va, "acc:", accuracy(P_va, y_va))


Step2(main) train LL: -6457.117020724935 acc: 0.6510270774976658
Step2(main) val   LL: -1635.6282220648823 acc: 0.6462110646211064


In [30]:
# Cell 6: prepare cluster bootstrap helpers

train_ids = df_train["ID"].unique()
print("num train respondents:", len(train_ids))

rng = np.random.default_rng(2025)

def resample_by_id(df, ids, rng):
    # 有放回抽样 respondent IDs
    sampled_ids = rng.choice(ids, size=len(ids), replace=True)
    # 拼接每个 respondent 的所有行
    df_b = pd.concat([df[df["ID"] == i] for i in sampled_ids], ignore_index=True)
    return df_b, sampled_ids


num train respondents: 952


In [31]:
# Cell 7: one bootstrap replicate (B=1 sanity)

df_b, sampled_ids = resample_by_id(df_train, train_ids, rng)

res_b = fit_step2_main(df_b, tt_scale, co_scale, he_scale, theta_init=theta2_main, maxiter=5000, maxfun=5000)

print("bootstrap one-run success:", res_b.success, res_b.message)
print("negLL(b):", res_b.fun)


bootstrap one-run success: False STOP: TOTAL NO. OF F,G EVALUATIONS EXCEEDS LIMIT
negLL(b): 6241.382485012706


In [32]:
# Cell 8: one bootstrap replicate with larger budget

df_b, sampled_ids = resample_by_id(df_train, train_ids, rng)

res_b = fit_step2_main(
    df_b,
    tt_scale, co_scale, he_scale,
    theta_init=theta2_main,
    maxiter=20000,
    maxfun=20000
)

print("bootstrap one-run success:", res_b.success, res_b.message)
print("negLL(b):", res_b.fun)


bootstrap one-run success: True CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
negLL(b): 6471.41272204629


In [33]:
def cluster_bootstrap_thetas(df_train, train_ids, B=30, seed=2025,
                            maxiter=20000, maxfun=20000,
                            theta_start=None, verbose_every=5):
    rng = np.random.default_rng(seed)
    thetas = []
    kept = 0

    for b in range(1, B+1):
        # 1) resample respondents with replacement
        df_b, _ = resample_by_id(df_train, train_ids, rng)

        # 2) fit with warm start
        res_b = fit_step2_main(
            df_b,
            tt_scale, co_scale, he_scale,
            theta_init=(theta_start if theta_start is not None else theta2_main),
            maxiter=maxiter, maxfun=maxfun
        )

        if res_b.success:
            thetas.append(res_b.x.copy())
            kept += 1
            theta_start = res_b.x  # 用本轮结果做下一轮 warm start（更稳）
        else:
            # 不成功就继续用上一轮的 theta_start（不要用全0）
            pass

        if (b % verbose_every) == 0:
            print(f"[bootstrap {b}/{B}] kept_success={kept}")

    thetas = np.array(thetas) if kept > 0 else np.zeros((0, len(theta2_main)))
    return thetas

thetas_boot = cluster_bootstrap_thetas(df_train, train_ids, B=30, seed=2025, verbose_every=5)

print("boot kept:", thetas_boot.shape[0], "theta dim:", (thetas_boot.shape[1] if thetas_boot.shape[0]>0 else None))


[bootstrap 5/30] kept_success=5
[bootstrap 10/30] kept_success=8
[bootstrap 15/30] kept_success=13
[bootstrap 20/30] kept_success=17
[bootstrap 25/30] kept_success=20
[bootstrap 30/30] kept_success=23
boot kept: 23 theta dim: 42


In [34]:
# Cell 11a: build param names that match theta2_main exactly

K_main = (len(theta2_main) - 6) // 2
print("theta2_main dim:", len(theta2_main), "=> K_main:", K_main)

X_names_all = get_X_names()   # 你之前用于打印 gamma 的那套名字
# Step2(main) 不含 FIRST，所以这里必须去掉 FIRST
X_names_main = [n for n in X_names_all if n != "FIRST"]

print("len(X_names_all):", len(X_names_all))
print("len(X_names_main) after drop FIRST:", len(X_names_main))

# 万一 get_X_names() 里不叫 FIRST 或还有别的多余项，做一个保险裁剪到 K_main
if len(X_names_main) != K_main:
    print("WARNING: X_names_main length != K_main, will truncate to K_main")
    X_names_main = X_names_main[:K_main]

param_names = (
    ["B_TT","B_CO","B_HE","B_SEATS","ASC_SM","ASC_CAR"]
    + [f"G_SM:{n}"  for n in X_names_main]
    + [f"G_CAR:{n}" for n in X_names_main]
)

print("len(param_names):", len(param_names))
assert len(param_names) == len(theta2_main), (len(param_names), len(theta2_main))


theta2_main dim: 42 => K_main: 18
len(X_names_all): 19
len(X_names_main) after drop FIRST: 18
len(param_names): 42


In [35]:
# Cell 11b: build bootstrap SE table

import pandas as pd
import numpy as np
from scipy.stats import norm

se_boot = thetas_boot.std(axis=0, ddof=1)
print("se_boot shape:", se_boot.shape)

# 防止极小 SE 导致除零
eps = 1e-12
z = theta2_main / (se_boot + eps)
p = 2 * (1 - norm.cdf(np.abs(z)))

def sig_star(p):
    if p < 0.001: return "***"
    if p < 0.01:  return "**"
    if p < 0.05:  return "*"
    if p < 0.1:   return "."
    return ""

df_boot = pd.DataFrame({
    "param": param_names,
    "coef": theta2_main,
    "boot_se": se_boot,
    "z": z,
    "p": p
})
df_boot["sig"] = df_boot["p"].apply(sig_star)

print("Head params:")
display(df_boot.iloc[:6])

print("Top by |z|:")
display(df_boot.reindex(df_boot["z"].abs().sort_values(ascending=False).index).head(20))


se_boot shape: (42,)
Head params:


,param,coef,boot_se,z,p,sig
0,B_TT,-0.707356,0.173778,-4.070455,4.692136e-05,***
1,B_CO,-0.251351,0.036340,-6.916593,4.626299e-12,***
2,B_HE,-0.123693,0.019645,-6.296534,3.043739e-10,***
3,B_SEATS,0.083442,0.098365,0.848296,3.962731e-01,
4,ASC_SM,1.649481,0.683494,2.413306,1.580854e-02,*
5,ASC_CAR,0.068880,0.966411,0.071274,9.431798e-01,


Top by |z|:


,param,coef,boot_se,z,p,sig
37,G_CAR:PURPOSE_5,-7.796485,0.379241,-20.558102,0.000000e+00,***
40,G_CAR:PURPOSE_8,-7.152697,0.432395,-16.542048,0.000000e+00,***
22,G_SM:PURPOSE_8,-2.812230,0.181978,-15.453660,0.000000e+00,***
1,B_CO,-0.251351,0.036340,-6.916593,4.626299e-12,***
2,B_HE,-0.123693,0.019645,-6.296534,3.043739e-10,***
36,G_CAR:PURPOSE_4,1.441201,0.313523,4.596795,4.290388e-06,***
0,B_TT,-0.707356,0.173778,-4.070455,4.692136e-05,***
28,G_CAR:AGE_5,-1.880744,0.556732,-3.378187,7.296534e-04,***
11,G_SM:AGE_6,0.734817,0.218726,3.359532,7.807447e-04,***
14,G_SM:INCOME_3,-1.014318,0.303589,-3.341092,8.344964e-04,***


In [36]:
# Cell: Cluster bootstrap (by ID) for Step2(main) with larger B + retry-on-fail

import numpy as np

B = 200                      # 你可以先 100，再 200；现在按 200 来
seed = 2026
rng = np.random.default_rng(seed)

thetas = []
fail = 0

for b in range(B):
    # 1) 按 ID 重采样（cluster bootstrap）
    df_b, sampled_ids = resample_by_id(df_train, train_ids, rng)

    # 2) 拟合 Step2(main)：先用 theta2_main 当初值
    res_b = fit_step2_main(
        df_b, tt_scale, co_scale, he_scale,
        theta_init=theta2_main,
        maxiter=20000, maxfun=20000
    )

    # 3) 若没收敛：用本次 res_b.x 作为“更贴近”的初值再跑一次（救回不少 kept）
    if (not res_b.success) and (hasattr(res_b, "x")) and np.all(np.isfinite(res_b.x)):
        res_b2 = fit_step2_main(
            df_b, tt_scale, co_scale, he_scale,
            theta_init=res_b.x,
            maxiter=20000, maxfun=20000
        )
        res_b = res_b2

    # 4) 只保留真正收敛且参数有限的结果
    if res_b.success and np.all(np.isfinite(res_b.x)):
        thetas.append(res_b.x.copy())
    else:
        fail += 1

    if (b + 1) % 20 == 0:
        print(f"[bootstrap {b+1}/{B}] kept={len(thetas)} fail={fail}")

thetas_boot = np.vstack(thetas) if len(thetas) > 0 else np.empty((0, len(theta2_main)))
print("boot kept:", thetas_boot.shape[0], "theta dim:", thetas_boot.shape[1])

# 先只做到这里：确认 kept > 0 且维度=42


[bootstrap 20/200] kept=20 fail=0
[bootstrap 40/200] kept=40 fail=0
[bootstrap 60/200] kept=60 fail=0
[bootstrap 80/200] kept=80 fail=0
[bootstrap 100/200] kept=100 fail=0
[bootstrap 120/200] kept=120 fail=0
[bootstrap 140/200] kept=140 fail=0
[bootstrap 160/200] kept=160 fail=0
[bootstrap 180/200] kept=180 fail=0
[bootstrap 200/200] kept=200 fail=0
boot kept: 200 theta dim: 42


In [37]:
import numpy as np
import pandas as pd
from scipy.stats import norm

# ---- 1) 生成与 Step2(main) 一致的 X_names_main（18个）----
def get_X_names_all():
    # 必须与你 build_X_ind / onehot levels 完全一致
    names = []
    names.append("FIRST")
    names.append("MALE")
    # AGE levels=[1..6], drop_first=True -> 2..6
    names += [f"AGE_{k}" for k in [2,3,4,5,6]]
    # INCOME levels=[0..4], drop_first=True -> 1..4
    names += [f"INCOME_{k}" for k in [1,2,3,4]]
    # PURPOSE levels=[1..9], drop_first=True -> 2..9
    names += [f"PURPOSE_{k}" for k in [2,3,4,5,6,7,8,9]]
    return names

X_names_all  = get_X_names_all()
X_names_main = [n for n in X_names_all if n != "FIRST"]   # Step2(main) 不含 FIRST
K_main = len(X_names_main)

print("K_main:", K_main, "| theta2_main dim:", len(theta2_main), "| thetas_boot dim:", thetas_boot.shape[1])

# ---- 2) 参数名（必须=42）----
head_names = ["B_TT","B_CO","B_HE","B_SEATS","ASC_SM","ASC_CAR"]
param_names = head_names \
    + [f"G_SM:{n}" for n in X_names_main] \
    + [f"G_CAR:{n}" for n in X_names_main]

assert len(param_names) == len(theta2_main) == thetas_boot.shape[1], (len(param_names), len(theta2_main), thetas_boot.shape[1])

# ---- 3) Cluster bootstrap SE / z / p / CI ----
boot_se = thetas_boot.std(axis=0, ddof=1)

# 95% percentile CI（最常用、最直观）
ci_lo = np.percentile(thetas_boot, 2.5, axis=0)
ci_hi = np.percentile(thetas_boot, 97.5, axis=0)

coef = np.asarray(theta2_main)
z = np.divide(coef, boot_se, out=np.full_like(coef, np.nan), where=(boot_se>0))
p = 2 * (1 - norm.cdf(np.abs(z)))

def stars(pv):
    if pv < 0.001: return "***"
    if pv < 0.01:  return "**"
    if pv < 0.05:  return "*"
    if pv < 0.1:   return "."
    return ""

sig = [stars(x) for x in p]

df_boot = pd.DataFrame({
    "param": param_names,
    "coef": coef,
    "boot_se": boot_se,
    "z": z,
    "p": p,
    "sig": sig,
    "ci_2.5": ci_lo,
    "ci_97.5": ci_hi
})

# ---- 4) 输出你最关心的两块 ----
print("\n=== Head params (6) ===")
display(df_boot.iloc[:6].copy())

df_top = df_boot.copy()
df_top["abs_z"] = np.abs(df_top["z"])
df_top = df_top.sort_values("abs_z", ascending=False).drop(columns="abs_z")

print("\n=== Top 20 by |z| (bootstrap) ===")
display(df_top.head(20))


K_main: 18 | theta2_main dim: 42 | thetas_boot dim: 42

=== Head params (6) ===


,param,coef,boot_se,z,p,sig,ci_2.5,ci_97.5
0,B_TT,-0.707356,0.139460,-5.072123,3.934018e-07,***,-0.974216,-0.449283
1,B_CO,-0.251351,0.035255,-7.129449,1.007638e-12,***,-0.324146,-0.194891
2,B_HE,-0.123693,0.017233,-7.177845,7.081002e-13,***,-0.157160,-0.088173
3,B_SEATS,0.083442,0.109385,0.762828,4.455658e-01,,-0.119165,0.338899
4,ASC_SM,1.649481,0.832591,1.981142,4.757531e-02,*,0.506101,3.325639
5,ASC_CAR,0.068880,0.936933,0.073516,9.413952e-01,,-1.285947,2.062669



=== Top 20 by |z| (bootstrap) ===


,param,coef,boot_se,z,p,sig,ci_2.5,ci_97.5
22,G_SM:PURPOSE_8,-2.812230,0.178529,-15.752200,0.000000e+00,***,-3.288633,-2.558179
37,G_CAR:PURPOSE_5,-7.796485,0.697096,-11.184235,0.000000e+00,***,-10.459663,-7.804425
40,G_CAR:PURPOSE_8,-7.152697,0.941550,-7.596724,3.042011e-14,***,-10.300911,-7.152697
2,B_HE,-0.123693,0.017233,-7.177845,7.081002e-13,***,-0.157160,-0.088173
1,B_CO,-0.251351,0.035255,-7.129449,1.007638e-12,***,-0.324146,-0.194891
0,B_TT,-0.707356,0.139460,-5.072123,3.934018e-07,***,-0.974216,-0.449283
36,G_CAR:PURPOSE_4,1.441201,0.300122,4.802049,1.570505e-06,***,0.899468,2.013039
14,G_SM:INCOME_3,-1.014318,0.294164,-3.448140,5.644610e-04,***,-1.584620,-0.456561
11,G_SM:AGE_6,0.734817,0.227741,3.226551,1.252919e-03,**,0.265239,1.126739
6,G_SM:MALE,0.496189,0.165305,3.001653,2.685182e-03,**,0.139320,0.810429


In [38]:
import numpy as np
import pandas as pd

# 你应该已经有：
# theta2_main (42,)
# se_boot (42,)
# thetas_boot shape: (B_kept, 42)
# K_main = 18
# X_names_all / X_names_main 其中之一（主模型不含 FIRST）

# --- 如果你已经有 X_names_main，就跳过这段自动构建 ---
try:
    X_names_main
except NameError:
    # 如果你只有 X_names_all（19 个，含 FIRST），这里自动 drop FIRST
    # 你之前打印过：len(X_names_all)=19, len(X_names_main)=18
    X_names_main = [n for n in X_names_all if n != "FIRST"]

assert len(X_names_main) == K_main, (len(X_names_main), K_main)

param_names = (
    ["B_TT","B_CO","B_HE","B_SEATS","ASC_SM","ASC_CAR"]
    + [f"G_SM:{n}"  for n in X_names_main]
    + [f"G_CAR:{n}" for n in X_names_main]
)

print("len(param_names):", len(param_names))
assert len(param_names) == len(theta2_main) == len(se_boot) == 6 + 2*K_main


len(param_names): 42


In [39]:
# thetas_boot: (B_kept, 42)
boot_lo = np.percentile(thetas_boot, 2.5, axis=0)
boot_hi = np.percentile(thetas_boot, 97.5, axis=0)

z = theta2_main / se_boot
# 正态近似 p（论文里可以用；更严格也可以给 bootstrap p，下一 cell 给）
from scipy.stats import norm
p_norm = 2 * (1 - norm.cdf(np.abs(z)))

df_res = pd.DataFrame({
    "param": param_names,
    "coef": theta2_main,
    "boot_se": se_boot,
    "z": z,
    "p_norm": p_norm,
    "ci_2.5": boot_lo,
    "ci_97.5": boot_hi,
})

def sig_star(p):
    if p < 0.01: return "***"
    if p < 0.05: return "**"
    if p < 0.1:  return "*"
    return ""

df_res["sig"] = df_res["p_norm"].apply(sig_star)

df_res.head(10)


/tmp/ipython-input-1741482414.py:5: RuntimeWarning: invalid value encountered in divide
  z = theta2_main / se_boot


,param,coef,boot_se,z,p_norm,ci_2.5,ci_97.5,sig
0,B_TT,-0.707356,0.173778,-4.070455,4.692136e-05,-0.974216,-0.449283,***
1,B_CO,-0.251351,0.036340,-6.916593,4.626299e-12,-0.324146,-0.194891,***
2,B_HE,-0.123693,0.019645,-6.296534,3.043739e-10,-0.157160,-0.088173,***
3,B_SEATS,0.083442,0.098365,0.848296,3.962731e-01,-0.119165,0.338899,
4,ASC_SM,1.649481,0.683494,2.413306,1.580854e-02,0.506101,3.325639,**
5,ASC_CAR,0.068880,0.966411,0.071274,9.431798e-01,-1.285947,2.062669,
6,G_SM:MALE,0.496189,0.160349,3.094434,1.971889e-03,0.139320,0.810429,***
7,G_SM:AGE_2,-1.317077,0.539379,-2.441840,1.461261e-02,-2.981862,-0.271338,**
8,G_SM:AGE_3,-0.837446,0.480792,-1.741806,8.154240e-02,-2.448148,0.082448,*
9,G_SM:AGE_4,-0.492281,0.439005,-1.121356,2.621362e-01,-2.122857,0.568397,


In [40]:
# 以“对称双尾”的 bootstrap p（推荐写法）
# p_boot = 2*min(P(theta* <= 0), P(theta* >= 0))
p_left  = np.mean(thetas_boot <= 0, axis=0)
p_right = np.mean(thetas_boot >= 0, axis=0)
p_boot = 2 * np.minimum(p_left, p_right)
p_boot = np.clip(p_boot, 0, 1)

df_res["p_boot"] = p_boot
df_res["sig_boot"] = df_res["p_boot"].apply(sig_star)

# 看看主干系数（前 6 个）
df_res.iloc[:6][["param","coef","boot_se","p_boot","sig_boot","ci_2.5","ci_97.5"]]


,param,coef,boot_se,p_boot,sig_boot,ci_2.5,ci_97.5
0,B_TT,-0.707356,0.173778,0.00,***,-0.974216,-0.449283
1,B_CO,-0.251351,0.036340,0.00,***,-0.324146,-0.194891
2,B_HE,-0.123693,0.019645,0.00,***,-0.157160,-0.088173
3,B_SEATS,0.083442,0.098365,0.45,,-0.119165,0.338899
4,ASC_SM,1.649481,0.683494,0.01,**,0.506101,3.325639
5,ASC_CAR,0.068880,0.966411,0.85,,-1.285947,2.062669


In [41]:
K = K_main
idx_head = np.arange(0, 6)
idx_gsm  = np.arange(6, 6+K)
idx_gcar = np.arange(6+K, 6+2*K)

df_head = df_res.iloc[idx_head].copy()
df_gsm  = df_res.iloc[idx_gsm].copy()
df_gcar = df_res.iloc[idx_gcar].copy()

print("=== Head params (6) ===")
display(df_head[["param","coef","boot_se","p_boot","sig_boot","ci_2.5","ci_97.5"]])

# Top 20 by |z|（用 bootstrap se 算 z）
df_top = df_res.copy()
df_top["abs_z"] = np.abs(df_top["z"])
df_top = df_top.sort_values("abs_z", ascending=False).head(20)

print("=== Top 20 by |z| (bootstrap se) ===")
display(df_top[["param","coef","boot_se","z","p_boot","sig_boot","ci_2.5","ci_97.5"]])


=== Head params (6) ===


,param,coef,boot_se,p_boot,sig_boot,ci_2.5,ci_97.5
0,B_TT,-0.707356,0.173778,0.00,***,-0.974216,-0.449283
1,B_CO,-0.251351,0.036340,0.00,***,-0.324146,-0.194891
2,B_HE,-0.123693,0.019645,0.00,***,-0.157160,-0.088173
3,B_SEATS,0.083442,0.098365,0.45,,-0.119165,0.338899
4,ASC_SM,1.649481,0.683494,0.01,**,0.506101,3.325639
5,ASC_CAR,0.068880,0.966411,0.85,,-1.285947,2.062669


=== Top 20 by |z| (bootstrap se) ===


,param,coef,boot_se,z,p_boot,sig_boot,ci_2.5,ci_97.5
37,G_CAR:PURPOSE_5,-7.796485,0.379241,-20.558102,0.00,***,-10.459663,-7.804425
40,G_CAR:PURPOSE_8,-7.152697,0.432395,-16.542048,0.00,***,-10.300911,-7.152697
22,G_SM:PURPOSE_8,-2.812230,0.181978,-15.453660,0.00,***,-3.288633,-2.558179
1,B_CO,-0.251351,0.036340,-6.916593,0.00,***,-0.324146,-0.194891
2,B_HE,-0.123693,0.019645,-6.296534,0.00,***,-0.157160,-0.088173
36,G_CAR:PURPOSE_4,1.441201,0.313523,4.596795,0.00,***,0.899468,2.013039
0,B_TT,-0.707356,0.173778,-4.070455,0.00,***,-0.974216,-0.449283
28,G_CAR:AGE_5,-1.880744,0.556732,-3.378187,0.00,***,-3.509062,-0.789601
11,G_SM:AGE_6,0.734817,0.218726,3.359532,0.00,***,0.265239,1.126739
14,G_SM:INCOME_3,-1.014318,0.303589,-3.341092,0.00,***,-1.584620,-0.456561


In [42]:
def add_odds_ratio(df):
    out = df.copy()
    out["odds_ratio"] = np.exp(out["coef"])
    out["or_ci_2.5"]  = np.exp(out["ci_2.5"])
    out["or_ci_97.5"] = np.exp(out["ci_97.5"])
    return out

df_gsm_or  = add_odds_ratio(df_gsm)
df_gcar_or = add_odds_ratio(df_gcar)

print("=== Significant G_SM (p_boot<0.05) ===")
display(df_gsm_or[df_gsm_or["p_boot"]<0.05][["param","coef","odds_ratio","p_boot","or_ci_2.5","or_ci_97.5"]].sort_values("p_boot"))

print("=== Significant G_CAR (p_boot<0.05) ===")
display(df_gcar_or[df_gcar_or["p_boot"]<0.05][["param","coef","odds_ratio","p_boot","or_ci_2.5","or_ci_97.5"]].sort_values("p_boot"))


=== Significant G_SM (p_boot<0.05) ===


,param,coef,odds_ratio,p_boot,or_ci_2.5,or_ci_97.5
6,G_SM:MALE,0.496189,1.642450,0.00,1.149492,2.248872
11,G_SM:AGE_6,0.734817,2.085100,0.00,1.303743,3.085579
22,G_SM:PURPOSE_8,-2.812230,0.060071,0.00,0.037305,0.077446
14,G_SM:INCOME_3,-1.014318,0.362650,0.00,0.205026,0.633458
12,G_SM:INCOME_1,0.666824,1.948041,0.01,1.224568,2.974899
10,G_SM:AGE_5,-1.390054,0.249062,0.01,0.047591,0.719938
7,G_SM:AGE_2,-1.317077,0.267917,0.02,0.050698,0.762359
16,G_SM:PURPOSE_2,-0.555959,0.573522,0.03,0.361157,0.940140


=== Significant G_CAR (p_boot<0.05) ===


,param,coef,odds_ratio,p_boot,or_ci_2.5,or_ci_97.5
28,G_CAR:AGE_5,-1.880744,0.152477,0.00,0.029925,0.454026
37,G_CAR:PURPOSE_5,-7.796485,0.000411,0.00,0.000029,0.000408
36,G_CAR:PURPOSE_4,1.441201,4.225769,0.00,2.458295,7.486032
40,G_CAR:PURPOSE_8,-7.152697,0.000783,0.00,0.000034,0.000783
34,G_CAR:PURPOSE_2,-0.965109,0.380942,0.01,0.196772,0.673812
24,G_CAR:MALE,0.535676,1.708603,0.01,1.146320,2.698879
30,G_CAR:INCOME_1,1.038402,2.824700,0.02,1.098017,8.549790
29,G_CAR:AGE_6,0.996468,2.708697,0.02,1.127104,7.300282
25,G_CAR:AGE_2,-1.372312,0.253520,0.03,0.059437,0.750274


In [43]:
import numpy as np, pandas as pd

alt_names = ["Train","SM","Car"]
idx = {n:i for i,n in enumerate(param_names)}

b_tt    = theta2_main[idx["B_TT"]]
b_co    = theta2_main[idx["B_CO"]]
b_he    = theta2_main[idx["B_HE"]]
b_seats = theta2_main[idx["B_SEATS"]]

tt_s = np.asarray(tt_scale).reshape(-1)   # (3,)
co_s = np.asarray(co_scale).reshape(-1)   # (3,)
he_s = np.asarray(he_scale).reshape(-1)   # (3,)

# 原单位边际效用：每分钟 / 每CHF
mu_tt = b_tt / tt_s
mu_co = b_co / co_s

# headway 只对 Train/SM 有意义；Car 设 NaN
mu_he = np.array([b_he/he_s[0], b_he/he_s[1], np.nan], dtype=float)

# ✅ 正确的 VOT / WTP（去掉负号）
VOT_chf_per_hr = 60.0 * (mu_tt / mu_co)
WTP_headway_chf_per_hr = 60.0 * (mu_he / mu_co)

# seats dummy通常只作用在 SM（如果你在 SEATS 只给 SM 列赋值）
WTP_seat_SM_chf = - b_seats / mu_co[1]

df_fix = pd.DataFrame({
    "alt": alt_names,
    "mu_tt_per_min": mu_tt,
    "mu_co_per_chf": mu_co,
    "VOT_CHF_per_hr": VOT_chf_per_hr,
    "WTP_headway_CHF_per_hr": WTP_headway_chf_per_hr
})
display(df_fix)

print("WTP seat dummy (SM), CHF:", WTP_seat_SM_chf)


,alt,mu_tt_per_min,mu_co_per_chf,VOT_CHF_per_hr,WTP_headway_CHF_per_hr
0,Train,-0.008908,-0.003682,145.162476,53.850732
1,SM,-0.012525,-0.000178,4225.060638,5112.817461
2,Car,-0.007865,-0.004507,104.716572,NaN


WTP seat dummy (SM), CHF: 469.1111893241133


In [44]:
import numpy as np
import pandas as pd

# 你已有的量（按你现在notebook里已有变量名）
# res_null.fun  = negLL(train) for Null(ASC only)
# LL_tr_1, LL_va_1, acc_tr_1, acc_va_1  (Step1)
# LL_tr_2m, LL_va_2m, acc_tr_2m, acc_va_2m (Step2 main)
# theta1, theta2_main 已有

LL0_tr = -res_null.fun  # 注意：res_null.fun是negLL，所以LL是负号

rows = [
    ["Null (ASC only)",       2,  LL0_tr,        np.nan, np.nan,  np.nan, np.nan],
    ["Step1 (context only)",  len(theta1), LL_tr_1, LL_va_1, acc_tr_1, acc_va_1, 1 - (LL_tr_1/LL0_tr)],
    ["Step2 main",            len(theta2_main), LL_tr_2m, LL_va_2m, acc_tr_2m, acc_va_2m, 1 - (LL_tr_2m/LL0_tr)],
]
df_fit = pd.DataFrame(rows, columns=["Model","Params","Train LL","Val LL","Train acc","Val acc","McFadden rho^2 (train)"])
df_fit["Δ Val LL vs Step1"] = df_fit["Val LL"] - df_fit.loc[df_fit["Model"].str.contains("Step1"), "Val LL"].values[0]
df_fit


,Model,Params,Train LL,Val LL,Train acc,Val acc,McFadden rho^2 (train),Δ Val LL vs Step1
0,Null (ASC only),2,-7579.615255,NaN,NaN,NaN,NaN,NaN
1,Step1 (context only),6,-7063.978480,-1741.173121,0.604575,0.604835,0.068029,0.000000
2,Step2 main,42,-6457.928077,-1634.839074,0.651027,0.643887,0.147987,106.334047


In [45]:
import pandas as pd
import numpy as np

# 你现在已有：
# param_names: list length 42
# theta2_main: np.array length 42
# se_boot: np.array length 42
# ci_2_5, ci_97_5: np.array length 42
# p_boot: np.array length 42   (或你算出来的p值)

df_param = pd.DataFrame({
    "param": param_names,
    "coef": theta2_main,
    "boot_se": se_boot,
    "p_boot": p_boot,
    "ci_2.5": ci_lo,
    "ci_97.5": ci_hi
})
# 暂时注释掉此行，因为对于百分位数法 Bootstrap CI，点估计不严格在区间内是可能发生的。
# assert np.all(ci_lo <= theta2_main) and np.all(theta2_main <= ci_hi)

head = ["B_TT","B_CO","B_HE","B_SEATS","ASC_SM","ASC_CAR"]
df_head = df_param[df_param["param"].isin(head)].copy()
df_head

,param,coef,boot_se,p_boot,ci_2.5,ci_97.5
0,B_TT,-0.707356,0.173778,0.00,-0.974216,-0.449283
1,B_CO,-0.251351,0.036340,0.00,-0.324146,-0.194891
2,B_HE,-0.123693,0.019645,0.00,-0.157160,-0.088173
3,B_SEATS,0.083442,0.098365,0.45,-0.119165,0.338899
4,ASC_SM,1.649481,0.683494,0.01,0.506101,3.325639
5,ASC_CAR,0.068880,0.966411,0.85,-1.285947,2.062669


In [46]:
import numpy as np
import pandas as pd

theta = np.asarray(theta2_main).ravel()
ci_lo = np.asarray(ci_lo).ravel()
ci_hi = np.asarray(ci_hi).ravel()

# 1) 真正该 assert 的：维度一致 + 下界<=上界
assert theta.shape == ci_lo.shape == ci_hi.shape, (theta.shape, ci_lo.shape, ci_hi.shape)
assert np.all(ci_lo <= ci_hi), "Found ci_lo > ci_hi for some params."

# 2) 不强制包含点估计，只做诊断：有多少参数落在CI外
outside = (theta < ci_lo) | (theta > ci_hi)
print("outside-CI count:", int(outside.sum()), "/", len(theta))

df_out = pd.DataFrame({
    "param": param_names,
    "coef": theta,
    "ci_2.5": ci_lo,
    "ci_97.5": ci_hi,
    "gap": np.where(theta < ci_lo, ci_lo-theta,
                    np.where(theta > ci_hi, theta-ci_hi, 0.0))
})
display(df_out.loc[outside].sort_values("gap", ascending=False).head(20))


outside-CI count: 1 / 42


,param,coef,ci_2.5,ci_97.5,gap
37,G_CAR:PURPOSE_5,-7.796485,-10.459663,-7.804425,0.007941


In [47]:
import numpy as np
import pandas as pd
from scipy.stats import norm

theta = np.asarray(theta2_main).ravel()
se = np.asarray(se_boot).ravel()
ci_lo = np.asarray(ci_lo).ravel()
ci_hi = np.asarray(ci_hi).ravel()
p_boot = np.asarray(p_boot).ravel()

# z & normal-approx p (仅作参考；正式建议用 p_boot)
z = theta / se
p_norm = 2 * (1 - norm.cdf(np.abs(z)))

def star(p):
    if p < 0.01: return "***"
    if p < 0.05: return "**"
    if p < 0.10: return "*"
    return ""

df_final = pd.DataFrame({
    "param": param_names,
    "coef": theta,
    "boot_se": se,
    "z": z,
    "p_boot": p_boot,
    "p_norm(ref)": p_norm,
    "ci_2.5": ci_lo,
    "ci_97.5": ci_hi,
})
df_final["sig_boot"] = df_final["p_boot"].apply(star)

# 建议排序：先主参数，再 G_SM，再 G_CAR（更好读）
order_key = (
    df_final["param"].str.startswith("B_").astype(int)*0 +
    df_final["param"].str.startswith("ASC_").astype(int)*1 +
    df_final["param"].str.startswith("G_SM:").astype(int)*2 +
    df_final["param"].str.startswith("G_CAR:").astype(int)*3
)
df_final = df_final.assign(_k=order_key).sort_values(["_k","param"]).drop(columns="_k").reset_index(drop=True)

display(df_final)


/tmp/ipython-input-3652994815.py:12: RuntimeWarning: invalid value encountered in divide
  z = theta / se


,param,coef,boot_se,z,p_boot,p_norm(ref),ci_2.5,ci_97.5,sig_boot
0,B_CO,-0.251351,0.036340,-6.916593,0.00,4.626299e-12,-0.324146,-0.194891,***
1,B_HE,-0.123693,0.019645,-6.296534,0.00,3.043739e-10,-0.157160,-0.088173,***
2,B_SEATS,0.083442,0.098365,0.848296,0.45,3.962731e-01,-0.119165,0.338899,
3,B_TT,-0.707356,0.173778,-4.070455,0.00,4.692136e-05,-0.974216,-0.449283,***
4,ASC_CAR,0.068880,0.966411,0.071274,0.85,9.431798e-01,-1.285947,2.062669,
5,ASC_SM,1.649481,0.683494,2.413306,0.01,1.580854e-02,0.506101,3.325639,**
6,G_SM:AGE_2,-1.317077,0.539379,-2.441840,0.02,1.461261e-02,-2.981862,-0.271338,**
7,G_SM:AGE_3,-0.837446,0.480792,-1.741806,0.08,8.154240e-02,-2.448148,0.082448,*
8,G_SM:AGE_4,-0.492281,0.439005,-1.121356,0.32,2.621362e-01,-2.122857,0.568397,
9,G_SM:AGE_5,-1.390054,0.541174,-2.568592,0.01,1.021126e-02,-3.045105,-0.328591,**


In [48]:
df_final.to_csv("swissmetro_step2main_bootstrap_results.csv", index=False)
print("saved: swissmetro_step2main_bootstrap_results.csv")


saved: swissmetro_step2main_bootstrap_results.csv


In [49]:
import pandas as pd

# 你现在应当已经有 df_param（整表，42行）
# df_param.columns 里至少应包含：param, coef, boot_se, p_boot, ci_2.5, ci_97.5
# 如果没有 sig_boot 也没关系（下面会自动处理）

df = df_param.copy()

# ---- 统一列顺序（有的就放，没有就跳过）----
preferred_cols = ["param", "coef", "boot_se", "p_boot", "ci_2.5", "ci_97.5", "sig_boot"]
cols = [c for c in preferred_cols if c in df.columns]
df = df[cols + [c for c in df.columns if c not in cols]]

# ========= Table 1：核心 6 个参数 =========
head = ["B_TT", "B_CO", "B_HE", "B_SEATS", "ASC_SM", "ASC_CAR"]
df_table1 = df[df["param"].isin(head)].copy()
df_table1["__order"] = df_table1["param"].map({p:i for i,p in enumerate(head)})
df_table1 = df_table1.sort_values("__order").drop(columns="__order")

# ========= Table 2：显著的异质性项（SM / CAR 分开）=========
# 口径：p_boot < 0.05
alpha = 0.05

df_sm = df[df["param"].astype(str).str.startswith("G_SM:")].copy()
df_car = df[df["param"].astype(str).str.startswith("G_CAR:")].copy()

df_table2_sm = df_sm[df_sm["p_boot"] < alpha].copy().sort_values("p_boot")
df_table2_car = df_car[df_car["p_boot"] < alpha].copy().sort_values("p_boot")

# ========= Appendix A1：全 42 参数（把 head 放最上面，其余按 param 排序）=========
df_appendix = df.copy()
df_appendix["__is_head"] = df_appendix["param"].isin(head).astype(int)
df_appendix["__head_order"] = df_appendix["param"].map({p:i for i,p in enumerate(head)})
df_appendix = df_appendix.sort_values(
    by=["__is_head", "__head_order", "param"],
    ascending=[False, True, True]
).drop(columns=["__is_head", "__head_order"])

# ========= 导出 CSV（utf-8-sig：Excel 直接打开不乱码）=========
df_table1.to_csv("Table1_core_params.csv", index=False, encoding="utf-8-sig")
df_table2_sm.to_csv("Table2_sig_interactions_SM.csv", index=False, encoding="utf-8-sig")
df_table2_car.to_csv("Table2_sig_interactions_CAR.csv", index=False, encoding="utf-8-sig")
df_appendix.to_csv("Appendix_A1_full_params.csv", index=False, encoding="utf-8-sig")

print("Saved:")
print(" - Table1_core_params.csv")
print(" - Table2_sig_interactions_SM.csv")
print(" - Table2_sig_interactions_CAR.csv")
print(" - Appendix_A1_full_params.csv")


Saved:
 - Table1_core_params.csv
 - Table2_sig_interactions_SM.csv
 - Table2_sig_interactions_CAR.csv
 - Appendix_A1_full_params.csv


In [50]:
from google.colab import files
files.download("Table1_core_params.csv")
files.download("Table2_sig_interactions_SM.csv")
files.download("Table2_sig_interactions_CAR.csv")
files.download("Appendix_A1_full_params.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [51]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# -----------------------------
# A) 读入你导出的 beta（Appendix_A1_full_params.csv）
# -----------------------------
def load_beta(beta_csv_path: str) -> dict:
    df = pd.read_csv(beta_csv_path)
    return dict(zip(df["param"].astype(str), df["coef"].astype(float)))

# -----------------------------
# B) 按你的 notebook：构造 TT/CO/HE/SEATS + 方案B GA处理 + /100
#    （这里直接对 DataFrame 做，不走 numpy dict，便于 synthetic 对接）
# -----------------------------
def build_alt_attributes(df: pd.DataFrame) -> dict:
    df = df.copy()
    # 必要列
    need = ["GA","TRAIN_TT","SM_TT","CAR_TT","TRAIN_CO","SM_CO","CAR_CO","TRAIN_HE","SM_HE","SM_SEATS"]
    miss = [c for c in need if c not in df.columns]
    if miss:
        raise ValueError(f"Missing columns: {miss}")

    N = len(df)
    GA = pd.to_numeric(df["GA"], errors="coerce").fillna(0).astype(int).to_numpy()

    # availability：若 synthetic 没给，默认都可用
    if "TRAIN_AV" not in df.columns: df["TRAIN_AV"] = 1
    if "SM_AV"    not in df.columns: df["SM_AV"]    = 1
    if "CAR_AV"   not in df.columns: df["CAR_AV"]   = 1
    AV = np.column_stack([
        pd.to_numeric(df["TRAIN_AV"], errors="coerce").fillna(1).to_numpy(float),
        pd.to_numeric(df["SM_AV"],    errors="coerce").fillna(1).to_numpy(float),
        pd.to_numeric(df["CAR_AV"],   errors="coerce").fillna(1).to_numpy(float),
    ])

    # TT (/100)
    TT = np.column_stack([
        pd.to_numeric(df["TRAIN_TT"], errors="coerce").to_numpy(float),
        pd.to_numeric(df["SM_TT"],    errors="coerce").to_numpy(float),
        pd.to_numeric(df["CAR_TT"],   errors="coerce").to_numpy(float),
    ]) / 100.0

    # CO 方案B：Train 票价 GA=1 -> 0；SM 不调；Car 原样，再 /100
    TRAIN_FARE = pd.to_numeric(df["TRAIN_CO"], errors="coerce").to_numpy(float) * (GA == 0)
    SM_COST    = pd.to_numeric(df["SM_CO"],    errors="coerce").to_numpy(float)
    CAR_COST   = pd.to_numeric(df["CAR_CO"],   errors="coerce").to_numpy(float)
    CO = np.column_stack([TRAIN_FARE, SM_COST, CAR_COST]) / 100.0

    # HE (/100)，Car=0
    HE = np.column_stack([
        pd.to_numeric(df["TRAIN_HE"], errors="coerce").to_numpy(float),
        pd.to_numeric(df["SM_HE"],    errors="coerce").to_numpy(float),
        np.zeros(N),
    ]) / 100.0

    # SEATS：仅 SM 有值
    SEATS = np.column_stack([np.zeros(N),
                             pd.to_numeric(df["SM_SEATS"], errors="coerce").fillna(0).to_numpy(float),
                             np.zeros(N)])

    return {"TT": TT, "CO": CO, "HE": HE, "SEATS": SEATS, "AV": AV}

# -----------------------------
# C) 生成 Step2(main) 的 X_ind（不含 FIRST，dummy 编码与导出一致）
# -----------------------------
def build_X_main(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for c in ["MALE","AGE","INCOME","PURPOSE"]:
        if c not in df.columns:
            raise ValueError(f"Missing column {c}")

    X = {}
    X["MALE"] = pd.to_numeric(df["MALE"], errors="coerce").fillna(0).astype(float)

    AGE = pd.to_numeric(df["AGE"], errors="coerce")
    for k in [2,3,4,5,6]:
        X[f"AGE_{k}"] = (AGE == k).astype(float)

    INC = pd.to_numeric(df["INCOME"], errors="coerce")
    for k in [1,2,3,4]:
        X[f"INCOME_{k}"] = (INC == k).astype(float)

    PUR = pd.to_numeric(df["PURPOSE"], errors="coerce")
    for k in [2,3,4,5,6,7,8,9]:
        X[f"PURPOSE_{k}"] = (PUR == k).astype(float)

    return pd.DataFrame(X)

# -----------------------------
# D) 计算缩放参数：严格复现 notebook（按 ID 切分 seed=42，在 train 上算 std）
# -----------------------------
def compute_scales_from_real(real_df: pd.DataFrame, seed=42, test_size=0.2) -> dict:
    df = real_df.copy()
    df = df.loc[df["CHOICE"] > 0].copy()

    ids = df["ID"].unique()
    train_ids, _ = train_test_split(ids, test_size=test_size, random_state=seed)
    df_train = df[df["ID"].isin(train_ids)].copy()

    mats = build_alt_attributes(df_train)
    tt_scale = mats["TT"].std(axis=0, keepdims=True) + 1e-12
    co_scale = mats["CO"].std(axis=0, keepdims=True) + 1e-12
    he_scale = mats["HE"].std(axis=0, keepdims=True) + 1e-12

    return {"tt_scale": tt_scale, "co_scale": co_scale, "he_scale": he_scale, "train_ids": train_ids}

# -----------------------------
# E) Softmax with availability（与你 notebook 的 softmax_with_av 同口径）
# -----------------------------
def softmax_with_av(U: np.ndarray, AV: np.ndarray, large_neg=-1e9) -> np.ndarray:
    U2 = U.copy()
    U2[AV <= 0] = large_neg
    Umax = np.max(U2, axis=1, keepdims=True)
    expU = np.exp(U2 - Umax)
    expU[AV <= 0] = 0.0
    denom = np.sum(expU, axis=1, keepdims=True) + 1e-300
    return expU / denom

# -----------------------------
# F) 评分：输出 P 与下游指标（avg_P_chosen / low_prob_rate / LL / acc）
# -----------------------------
def score_with_baseline_mnl(df: pd.DataFrame, beta: dict, scales: dict) -> pd.DataFrame:
    mats = build_alt_attributes(df)
    X = build_X_main(df)

    # scaled attributes
    TTs = mats["TT"] / scales["tt_scale"]
    COs = mats["CO"] / scales["co_scale"]
    HEs = mats["HE"] / scales["he_scale"]
    SEATS = mats["SEATS"]
    AV = mats["AV"]

    # head params
    b_tt    = float(beta.get("B_TT", 0.0))
    b_co    = float(beta.get("B_CO", 0.0))
    b_he    = float(beta.get("B_HE", 0.0))
    b_seats = float(beta.get("B_SEATS", 0.0))
    asc_sm  = float(beta.get("ASC_SM", 0.0))
    asc_car = float(beta.get("ASC_CAR", 0.0))

    U = b_tt*TTs + b_co*COs + b_he*HEs + b_seats*SEATS  # (N,3)

    # add ASC + interactions to SM/CAR only
    # 交互项在 beta 中以 G_SM:xxx / G_CAR:xxx 命名
    # 确保列存在：若 beta 有但 X 里没有，直接报错（防止悄悄对不上）
    for p, c in beta.items():
        if p.startswith("G_SM:"):
            name = p.split("G_SM:")[1]
            if name not in X.columns:
                raise ValueError(f"beta has {p} but X missing column {name}")
            U[:,1] += float(c) * X[name].to_numpy(float)
        elif p.startswith("G_CAR:"):
            name = p.split("G_CAR:")[1]
            if name not in X.columns:
                raise ValueError(f"beta has {p} but X missing column {name}")
            U[:,2] += float(c) * X[name].to_numpy(float)

    U[:,1] += asc_sm
    U[:,2] += asc_car

    P = softmax_with_av(U, AV)

    out = df.copy()
    out["P_TRAIN"] = P[:,0]
    out["P_SM"]    = P[:,1]
    out["P_CAR"]   = P[:,2]
    out["PRED_CHOICE"] = np.argmax(P, axis=1) + 1
    return out

def downstream_metrics(scored_df: pd.DataFrame, choice_col="CHOICE") -> dict:
    df = scored_df.copy()
    df[choice_col] = pd.to_numeric(df[choice_col], errors="coerce")
    df = df[df[choice_col].isin([1,2,3])].copy()
    if len(df) == 0:
        return {"N": 0}

    chosen_p = np.where(
        df[choice_col].to_numpy() == 1, df["P_TRAIN"].to_numpy(),
        np.where(df[choice_col].to_numpy() == 2, df["P_SM"].to_numpy(), df["P_CAR"].to_numpy())
    )
    ll = float(np.sum(np.log(np.clip(chosen_p, 1e-300, 1.0))))

    return {
        "N": int(len(df)),
        "avg_P_chosen": float(np.mean(chosen_p)),
        "low_prob_rate(<0.01)": float(np.mean(chosen_p < 0.01)),
        "ll_per_obs": float(ll / len(df)),
        "accuracy": float(np.mean(df["PRED_CHOICE"].to_numpy() == df[choice_col].to_numpy())),
        "pred_share_train": float(df["P_TRAIN"].mean()),
        "pred_share_sm": float(df["P_SM"].mean()),
        "pred_share_car": float(df["P_CAR"].mean()),
    }

# -----------------------------
# G) （可选）更“像论文”的 feasibility 快检
# -----------------------------
def feasibility_quickcheck(df: pd.DataFrame) -> dict:
    df = df.copy()
    issues = {}

    # 必要列（synthetic 最少要有这些才能被 scorer 吃下）
    required = ["ID","GA","MALE","AGE","INCOME","PURPOSE","CHOICE",
                "TRAIN_TT","SM_TT","CAR_TT","TRAIN_CO","SM_CO","CAR_CO","TRAIN_HE","SM_HE","SM_SEATS"]
    issues["missing_cols"] = [c for c in required if c not in df.columns]

    # 非法值：time/cost/headway 负数
    for c in ["TRAIN_TT","SM_TT","CAR_TT","TRAIN_CO","SM_CO","CAR_CO","TRAIN_HE","SM_HE"]:
        if c in df.columns:
            v = pd.to_numeric(df[c], errors="coerce")
            issues[f"neg_rate_{c}"] = float((v < 0).mean())

    # availability 与 choice 冲突（若提供了 AV）
    if "CAR_AV" in df.columns:
        car_av = pd.to_numeric(df["CAR_AV"], errors="coerce").fillna(1)
        ch = pd.to_numeric(df["CHOICE"], errors="coerce")
        issues["choice_car_while_unavailable_rate"] = float(((ch==3) & (car_av<=0)).mean())

    issues["row_na_rate"] = float(df.isna().any(axis=1).mean())
    issues["N"] = int(len(df))
    return issues


In [52]:
beta = load_beta("Appendix_A1_full_params.csv")
real = pd.read_csv("/content/drive/MyDrive/swissmetro/swissmetro.dat", sep=r"\s+", engine="python")

scales = compute_scales_from_real(real, seed=42, test_size=0.2)

real_scored = score_with_baseline_mnl(real, beta, scales)
print("real metrics:", downstream_metrics(real_scored))

# 若你有 synthetic_df：
# print("synth feasibility:", feasibility_quickcheck(synthetic_df))
# synth_scored = score_with_baseline_mnl(synthetic_df, beta, scales)
# print("synth metrics:", downstream_metrics(synth_scored))

real metrics: {'N': 10719, 'avg_P_chosen': 0.48279131959194727, 'low_prob_rate(<0.01)': 0.001119507416736636, 'll_per_obs': -0.8583514172748995, 'accuracy': 0.600429144509749, 'pred_share_train': 0.19288798145000363, 'pred_share_sm': 0.5191085117066861, 'pred_share_car': 0.28800350684331033}


In [53]:
def split_real_by_train_ids(real_df, train_ids):
    real_train = real_df[real_df["ID"].isin(train_ids)].copy()
    real_test  = real_df[~real_df["ID"].isin(train_ids)].copy()
    return real_train, real_test


In [54]:
COMBO_COLS = ["MALE","AGE","INCOME","PURPOSE","GA"]

def unique_combos(df, cols=COMBO_COLS):
    return df[cols].dropna().drop_duplicates()

def diversity_report(synth_df, real_train, real_test, real_all=None, cols=COMBO_COLS):
    if real_all is None:
        real_all = pd.concat([real_train, real_test], axis=0)

    S = unique_combos(synth_df, cols)
    T = unique_combos(real_train, cols)
    E = unique_combos(real_test, cols)
    A = unique_combos(real_all, cols)

    # 覆盖：synthetic 覆盖 train/test 的比例（关注对 test 的补覆盖）
    T_set = set(map(tuple, T.to_numpy()))
    E_set = set(map(tuple, E.to_numpy()))
    A_set = set(map(tuple, A.to_numpy()))
    S_set = set(map(tuple, S.to_numpy()))

    cover_train = len(S_set & T_set) / max(len(T_set), 1)
    cover_test  = len(S_set & E_set) / max(len(E_set), 1)

    # “新增覆盖”：synthetic 覆盖到了 test 里但 train 没见过的组合（这就是增样价值）
    unseen_test = E_set - T_set
    cover_unseen_test = len(S_set & unseen_test) / max(len(unseen_test), 1)

    # 结构零（precision）：synthetic 组合有多少真的出现在真实数据全集里
    precision_real = len(S_set & A_set) / max(len(S_set), 1)

    return {
        "uniq_combos_synth": int(len(S_set)),
        "uniq_combos_train": int(len(T_set)),
        "uniq_combos_test": int(len(E_set)),
        "cover_train_combo_rate": float(cover_train),
        "cover_test_combo_rate": float(cover_test),
        "cover_unseen_test_combo_rate": float(cover_unseen_test),
        "precision_combo_in_real_all": float(precision_real),
        "structural_zero_combo_rate": float(1 - precision_real),
        "n_unseen_test_combos": int(len(unseen_test)),
    }


In [55]:
def evaluate_one(synth_df, beta, scales, real_train, real_test, label="synth"):
    out = {"label": label, "N_synth": int(len(synth_df))}

    # 1) feasibility
    out.update({f"fz_{k}": v for k, v in feasibility_strict(synth_df).items() if k not in ["missing_cols"]})
    out["fz_missing_cols"] = str(feasibility_strict(synth_df).get("missing_cols", []))

    # 2) diversity
    out.update({f"dv_{k}": v for k, v in diversity_report(synth_df, real_train, real_test).items()})

    # 3) downstream consistency (baseline MNL scorer)
    scored = score_with_baseline_mnl(synth_df, beta, scales)
    out.update({f"ds_{k}": v for k, v in downstream_metrics(scored).items()})

    return out


In [56]:
# 已有：
beta = load_beta("Appendix_A1_full_params.csv")
real = pd.read_csv("/content/drive/MyDrive/swissmetro/swissmetro.dat", sep=r"\s+", engine="python")
scales = compute_scales_from_real(real, seed=42, test_size=0.2)
real_train, real_test = split_real_by_train_ids(real, scales["train_ids"])

# 评分：train/test 各一份
train_metrics = downstream_metrics(score_with_baseline_mnl(real_train, beta, scales))
test_metrics  = downstream_metrics(score_with_baseline_mnl(real_test,  beta, scales))

train_metrics, test_metrics

({'N': 8568,
  'avg_P_chosen': 0.4844036355248395,
  'low_prob_rate(<0.01)': 0.0014005602240896359,
  'll_per_obs': -0.8554740029665419,
  'accuracy': 0.6022408963585434,
  'pred_share_train': 0.19139948141838992,
  'pred_share_sm': 0.5171942701967672,
  'pred_share_car': 0.29140624838484297},
 {'N': 2151,
  'avg_P_chosen': 0.47636904022745574,
  'low_prob_rate(<0.01)': 0.0,
  'll_per_obs': -0.8698129169466843,
  'accuracy': 0.5932124593212459,
  'pred_share_train': 0.19881706944203806,
  'pred_share_sm': 0.5267334402315517,
  'pred_share_car': 0.27444949032641025})

In [57]:
# 先用 real_train 伪造一个 synthetic_df（debug 用）
synthetic_df = real_train.sample(n=2000, replace=True, random_state=0).copy()

print("synthetic_df shape:", synthetic_df.shape)
print("synthetic_df columns:", list(synthetic_df.columns)[:10], "...")


synthetic_df shape: (2000, 28)
synthetic_df columns: ['GROUP', 'SURVEY', 'SP', 'ID', 'PURPOSE', 'FIRST', 'TICKET', 'WHO', 'LUGGAGE', 'AGE'] ...


In [58]:
import pandas as pd
import numpy as np

# --- 最小 feasibility：只检查 required 列 + NA rate ---
def feasibility_min(df):
    required = ["ID","GA","MALE","AGE","INCOME","PURPOSE","CHOICE",
                "TRAIN_TT","SM_TT","CAR_TT","TRAIN_CO","SM_CO","CAR_CO",
                "TRAIN_HE","SM_HE","SM_SEATS"]
    missing = [c for c in required if c not in df.columns]
    return {
        "missing_cols": missing,
        "row_na_rate": float(df.isna().any(axis=1).mean()),
        "N": int(len(df))
    }

def evaluate_one_min(synth_df, beta, scales, label="synth"):
    out = {"label": label, "N_synth": int(len(synth_df))}

    # feasibility
    fz = feasibility_min(synth_df)
    out["fz_row_na_rate"] = fz["row_na_rate"]
    out["fz_missing_cols"] = str(fz["missing_cols"])

    # downstream
    scored = score_with_baseline_mnl(synth_df, beta, scales)
    ds = downstream_metrics(scored)
    for k,v in ds.items():
        out[f"ds_{k}"] = v

    return out

row = evaluate_one_min(synthetic_df, beta, scales, label="DEBUG_resample_real")
pd.DataFrame([row]).T


,0
label,DEBUG_resample_real
N_synth,2000
fz_row_na_rate,0.0
fz_missing_cols,[]
ds_N,2000
ds_avg_P_chosen,0.483794
ds_low_prob_rate(<0.01),0.002
ds_ll_per_obs,-0.863809
ds_accuracy,0.5995
ds_pred_share_train,0.186989


In [59]:
COMBO_COLS = ["MALE","AGE","INCOME","PURPOSE","GA"]

def unique_combos(df, cols=COMBO_COLS):
    return df[cols].dropna().drop_duplicates()

def diversity_report(synth_df, real_train, real_test, cols=COMBO_COLS):
    S = unique_combos(synth_df, cols)
    T = unique_combos(real_train, cols)
    E = unique_combos(real_test, cols)
    A = unique_combos(pd.concat([real_train, real_test], axis=0), cols)

    S_set = set(map(tuple, S.to_numpy()))
    T_set = set(map(tuple, T.to_numpy()))
    E_set = set(map(tuple, E.to_numpy()))
    A_set = set(map(tuple, A.to_numpy()))

    unseen_test = E_set - T_set

    precision_real = len(S_set & A_set) / max(len(S_set), 1)     # 结构零越低越好
    cover_train    = len(S_set & T_set) / max(len(T_set), 1)
    cover_test     = len(S_set & E_set) / max(len(E_set), 1)
    cover_unseen   = len(S_set & unseen_test) / max(len(unseen_test), 1)

    return {
        "uniq_combos_synth": int(len(S_set)),
        "cover_train_combo_rate": float(cover_train),
        "cover_test_combo_rate": float(cover_test),
        "cover_unseen_test_combo_rate": float(cover_unseen),
        "precision_combo_in_real_all": float(precision_real),
        "structural_zero_combo_rate": float(1 - precision_real),
        "n_unseen_test_combos": int(len(unseen_test)),
    }

def evaluate_one(synth_df, beta, scales, real_train, real_test, label="synth"):
    out = {"label": label, "N_synth": int(len(synth_df))}

    # feasibility（沿用你现在的最小版）
    fz = feasibility_min(synth_df)
    out["fz_row_na_rate"] = fz["row_na_rate"]
    out["fz_missing_cols"] = str(fz["missing_cols"])

    # diversity
    dv = diversity_report(synth_df, real_train, real_test)
    for k, v in dv.items():
        out[f"dv_{k}"] = v

    # downstream
    scored = score_with_baseline_mnl(synth_df, beta, scales)
    ds = downstream_metrics(scored)
    for k, v in ds.items():
        out[f"ds_{k}"] = v

    return out


In [60]:
row_full = evaluate_one(synthetic_df, beta, scales, real_train, real_test, label="DEBUG_resample_real")
pd.DataFrame([row_full]).T


,0
label,DEBUG_resample_real
N_synth,2000
fz_row_na_rate,0.0
fz_missing_cols,[]
dv_uniq_combos_synth,198
dv_cover_train_combo_rate,0.980198
dv_cover_test_combo_rate,0.660194
dv_cover_unseen_test_combo_rate,0.0
dv_precision_combo_in_real_all,1.0
dv_structural_zero_combo_rate,0.0


In [61]:
# 先从 real_train 抽 2000 条 task，当作 synthetic 的“情景”
syn_base = real_train.sample(n=2000, replace=True, random_state=1).copy()

# 用基准 MNL 计算概率并按概率抽 choice（模拟“一个很强的生成器”）
tmp = score_with_baseline_mnl(syn_base, beta, scales)
P = tmp[["P_TRAIN","P_SM","P_CAR"]].to_numpy()

rng = np.random.default_rng(1)
draw = [rng.choice([1,2,3], p=p) for p in P]
syn_base["CHOICE"] = draw

row = evaluate_one(syn_base, beta, scales, real_train, real_test, label="MNL_sampling_baseline")
pd.DataFrame([row]).T


,0
label,MNL_sampling_baseline
N_synth,2000
fz_row_na_rate,0.0
fz_missing_cols,[]
dv_uniq_combos_synth,195
dv_cover_train_combo_rate,0.965347
dv_cover_test_combo_rate,0.679612
dv_cover_unseen_test_combo_rate,0.0
dv_precision_combo_in_real_all,1.0
dv_structural_zero_combo_rate,0.0


In [62]:
COMBO_COLS = ["MALE","AGE","INCOME","PURPOSE","GA"]

train_set = set(map(tuple, real_train[COMBO_COLS].drop_duplicates().to_numpy()))
test_set  = set(map(tuple, real_test[COMBO_COLS].drop_duplicates().to_numpy()))

unseen_test = sorted(list(test_set - train_set))
print("n_unseen_test:", len(unseen_test))
unseen_test[:10]


n_unseen_test: 33


[(np.int64(0), np.int64(1), np.int64(1), np.int64(7), np.int64(0)),
 (np.int64(0), np.int64(2), np.int64(1), np.int64(9), np.int64(0)),
 (np.int64(0), np.int64(2), np.int64(2), np.int64(5), np.int64(1)),
 (np.int64(0), np.int64(2), np.int64(2), np.int64(7), np.int64(0)),
 (np.int64(0), np.int64(2), np.int64(3), np.int64(2), np.int64(1)),
 (np.int64(0), np.int64(2), np.int64(3), np.int64(4), np.int64(0)),
 (np.int64(0), np.int64(2), np.int64(4), np.int64(2), np.int64(1)),
 (np.int64(0), np.int64(3), np.int64(0), np.int64(2), np.int64(0)),
 (np.int64(0), np.int64(3), np.int64(4), np.int64(2), np.int64(1)),
 (np.int64(0), np.int64(4), np.int64(2), np.int64(1), np.int64(0))]

In [63]:
# 1) 抽一些任务当模板（保留 LOS 真实分布）
tmpl = real_train.sample(n=2000, replace=True, random_state=7).copy()

# 2) 把前 len(tmpl) 行的 combo 替换成 unseen_test combos（循环填充）
for i in range(len(tmpl)):
    combo = unseen_test[i % len(unseen_test)]
    tmpl.loc[tmpl.index[i], COMBO_COLS] = combo

# 3) 用基准 MNL 概率采样 CHOICE（像你之前的 MNL_sampling_baseline）
tmp_scored = score_with_baseline_mnl(tmpl, beta, scales)
P = tmp_scored[["P_TRAIN","P_SM","P_CAR"]].to_numpy()
rng = np.random.default_rng(7)
tmpl["CHOICE"] = [rng.choice([1,2,3], p=p) for p in P]

row = evaluate_one(tmpl, beta, scales, real_train, real_test, label="Target_unseen_combo + MNL_choice")
pd.DataFrame([row]).T


,0
label,Target_unseen_combo + MNL_choice
N_synth,2000
fz_row_na_rate,0.0
fz_missing_cols,[]
dv_uniq_combos_synth,33
dv_cover_train_combo_rate,0.0
dv_cover_test_combo_rate,0.320388
dv_cover_unseen_test_combo_rate,1.0
dv_precision_combo_in_real_all,1.0
dv_structural_zero_combo_rate,0.0


In [81]:
!pip -q install openai

import os, json, time
import numpy as np
import pandas as pd
from getpass import getpass
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = getpass("OPENAI_API_KEY: ")
client = OpenAI()


OPENAI_API_KEY: ··········


In [82]:
COMBO_COLS = ["MALE","AGE","INCOME","PURPOSE","GA"]

train_combos = list(set(map(tuple, real_train[COMBO_COLS].drop_duplicates().to_numpy())))
test_combos  = set(map(tuple, real_test[COMBO_COLS].drop_duplicates().to_numpy()))
unseen_test  = list(sorted(test_combos - set(train_combos)))

print("n_train_combos:", len(train_combos))
print("n_unseen_test:", len(unseen_test))

# 你可以调这个比例：补 unseen 的强度
p_unseen = 0.30
rng = np.random.default_rng(123)

def draw_combo():
    if len(unseen_test) > 0 and rng.random() < p_unseen:
        return unseen_test[rng.integers(len(unseen_test))]
    return train_combos[rng.integers(len(train_combos))]

N = 2000
templates = real_train.sample(n=N, replace=True, random_state=123).copy().reset_index(drop=True)

for i in range(N):
    templates.loc[i, COMBO_COLS] = draw_combo()

# 给 synthetic 新 ID，避免跟真实 respondent 混在一起
templates["ID"] = np.arange(1_000_000, 1_000_000 + N)


n_train_combos: 202
n_unseen_test: 33


In [83]:
schema = {
  "type": "object",
  "properties": {"CHOICE": {"type": "integer", "enum": [1,2,3]}},
  "required": ["CHOICE"],
  "additionalProperties": False
}

def make_prompt(row, cot=False):
    # cot=True 时给更强的“分步比较”提示，但仍只输出 JSON（schema 会强制）
    extra = ""
    if cot:
        extra = (
          "\nReasoning rubric (do internally): compare time first, then cost, then headway/comfort; "
          "avoid obviously dominated options."
        )

    return f"""You are simulating a discrete choice in Swissmetro. Output ONLY a JSON object that matches the schema.

Alternatives (encode choice as integer):
1=TRAIN, 2=SWISSMETRO, 3=CAR

Traveler attributes (coded):
MALE={int(row.MALE)}, AGE={int(row.AGE)}, INCOME={int(row.INCOME)}, PURPOSE={int(row.PURPOSE)}, GA={int(row.GA)}.

Level-of-service:
TRAIN: TT={float(row.TRAIN_TT)}, CO={float(row.TRAIN_CO)}, HE={float(row.TRAIN_HE)}
SM:    TT={float(row.SM_TT)},    CO={float(row.SM_CO)},    HE={float(row.SM_HE)}, SEATS={float(row.SM_SEATS)}
CAR:   TT={float(row.CAR_TT)},   CO={float(row.CAR_CO)}

Instruction:
- Choose the most preferred alternative for this traveler.
- Do NOT add any special preference for GA beyond its cost implication (no GA preference term).
- Return JSON with key CHOICE only.{extra}"""


In [84]:
MODEL_STAGE1 = "gpt-4o-mini"
jsonl1 = "/content/batch_stage1.jsonl"

with open(jsonl1, "w", encoding="utf-8") as f:
    for i in range(N):
        row = templates.loc[i]
        body = {
            "model": MODEL_STAGE1,
            "input": [
                {"role": "system", "content": "Return only strict JSON following the schema."},
                {"role": "user", "content": make_prompt(row, cot=False)}
            ],
            "text": {
                "format": {
                    "type": "json_schema",
                    "name": "choice_schema",
                    "strict": True,
                    "schema": schema
                }
            }
        }
        req = {"custom_id": f"req_{i}", "method": "POST", "url": "/v1/responses", "body": body}
        f.write(json.dumps(req) + "\n")

print("wrote:", jsonl1)


wrote: /content/batch_stage1.jsonl


In [85]:
from openai import OpenAI
client = OpenAI()

MODEL_STAGE1 = "gpt-4o-mini"

schema = {
  "type": "object",
  "properties": {"CHOICE": {"type": "integer", "enum": [1,2,3]}},
  "required": ["CHOICE"],
  "additionalProperties": False
}

fmt = {
  "type": "json_schema",
  "name": "choice_schema",   # ✅ 必填
  "strict": True,
  "schema": schema
}

row0 = templates.loc[0]
resp = client.responses.create(
    model=MODEL_STAGE1,
    input=[
        {"role":"system","content":"Return only strict JSON following the schema."},
        {"role":"user","content": make_prompt(row0, cot=False)}
    ],
    text={"format": fmt}
)

print(resp.output_text)


{"CHOICE":2}


In [86]:
MODEL_STAGE1 = "gpt-4o-mini"
jsonl1 = "/content/batch_stage1.jsonl"

fmt = {
  "type": "json_schema",
  "name": "choice_schema",   # ✅ 必填
  "strict": True,
  "schema": schema
}

with open(jsonl1, "w", encoding="utf-8") as f:
    for i in range(N):
        row = templates.loc[i]
        body = {
            "model": MODEL_STAGE1,
            "input": [
                {"role": "system", "content": "Return only strict JSON following the schema."},
                {"role": "user", "content": make_prompt(row, cot=False)}
            ],
            "text": {"format": fmt}   # ✅ 关键修复
        }
        req = {"custom_id": f"req_{i}", "method": "POST", "url": "/v1/responses", "body": body}
        f.write(json.dumps(req) + "\n")

print("wrote:", jsonl1)


wrote: /content/batch_stage1.jsonl


In [87]:
import time, json
from collections import Counter

TERMINAL = {"completed","failed","expired","cancelled"}

def _fmt_elapsed(sec):
    sec = int(sec)
    hh = sec // 3600
    mm = (sec % 3600) // 60
    ss = sec % 60
    return f"{hh:02d}:{mm:02d}:{ss:02d}"

def wait_batch_with_progress(batch_id, poll_s=15):
    start = time.time()
    last = None
    while True:
        b = client.batches.retrieve(batch_id)
        rc = getattr(b, "request_counts", None)
        prog = ""
        if rc is not None:
            prog = f" done {rc.completed}/{rc.total} | failed {rc.failed}"
        line = f"[{_fmt_elapsed(time.time()-start)}] {batch_id} -> {b.status}{prog}"
        if line != last:
            print(line)
            last = line
        if b.status in TERMINAL:
            return b
        time.sleep(poll_s)

def download_file_if_any(file_id, save_path):
    if not file_id:
        return None
    content = client.files.content(file_id)
    with open(save_path, "wb") as f:
        f.write(content.read())
    return save_path

def summarize_error_file(err_path, topk=5):
    ctr = Counter()
    with open(err_path, "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            err = obj.get("error") or {}
            code = err.get("code") or err.get("type") or "unknown"
            msg = (err.get("message") or "")[:120]
            ctr[(code, msg)] += 1
    print("\n--- Error summary (top) ---")
    for (code, msg), cnt in ctr.most_common(topk):
        print(f"{cnt}x  {code} | {msg}")

def wait_and_download_safe(batch_id, out_path, err_path=None, poll_s=15):
    b = wait_batch_with_progress(batch_id, poll_s=poll_s)
    out_id = getattr(b, "output_file_id", None)
    err_id = getattr(b, "error_file_id", None)
    if err_path is None:
        err_path = out_path.replace(".jsonl", "_errors.jsonl")

    print("\nBatch finished.")
    print("status:", b.status)
    print("output_file_id:", out_id)
    print("error_file_id:", err_id)

    out_saved = download_file_if_any(out_id, out_path)
    err_saved = download_file_if_any(err_id, err_path)

    if out_saved is None:
        if err_saved:
            summarize_error_file(err_saved, topk=8)
        raise RuntimeError("No output_file_id (no successful outputs). Check *_errors.jsonl")

    if err_saved:
        summarize_error_file(err_saved, topk=5)

    print("✅ Saved output:", out_saved)
    if err_saved:
        print("✅ Saved errors:", err_saved)
    return out_saved, err_saved

def submit_batch(jsonl_path, endpoint="/v1/responses"):
    up = client.files.create(file=open(jsonl_path, "rb"), purpose="batch")
    b = client.batches.create(
        input_file_id=up.id,
        endpoint=endpoint,
        completion_window="24h"
    )
    return b.id


In [89]:
from openai import OpenAI
client = OpenAI()

def debug_batch(batch_id):
    b = client.batches.retrieve(batch_id)
    print("batch_id:", b.id)
    print("status:", b.status)
    rc = getattr(b, "request_counts", None)
    print("request_counts:", rc)
    print("output_file_id:", getattr(b, "output_file_id", None))
    print("error_file_id :", getattr(b, "error_file_id", None))
    # 有些版本还有 status_details
    sd = getattr(b, "status_details", None)
    if sd is not None:
        print("status_details:", sd)
    return b

b = debug_batch(batch1_id)


batch_id: batch_696e42e5a3cc8190af90f8d7487cecdf
status: in_progress
request_counts: BatchRequestCounts(completed=0, failed=0, total=2000)
output_file_id: None
error_file_id : None


In [93]:
from openai import OpenAI
client = OpenAI()

bs = client.batches.list(limit=10)
for b in bs.data:
    rc = getattr(b, "request_counts", None)
    print(b.id, b.status, rc)


batch_696e47e2b80c81908c4402f0561f44be in_progress BatchRequestCounts(completed=0, failed=0, total=2000)
batch_696e46b37e148190bc31bf217a6f3935 in_progress BatchRequestCounts(completed=0, failed=0, total=500)
batch_696e42e5a3cc8190af90f8d7487cecdf cancelled BatchRequestCounts(completed=0, failed=0, total=2000)
batch_696e41df8b988190a38d3bb59136699f in_progress BatchRequestCounts(completed=0, failed=0, total=2000)
batch_696e398278288190829dda01fbfc2f3f in_progress BatchRequestCounts(completed=0, failed=0, total=2000)
batch_696e34454dac81908cc588b7cd997232 in_progress BatchRequestCounts(completed=0, failed=0, total=2000)
batch_696e0f3ec1708190b87930cbadffae5d completed BatchRequestCounts(completed=2000, failed=0, total=2000)
batch_696e07e871048190a9dc660ab755441c completed BatchRequestCounts(completed=2000, failed=0, total=2000)
batch_696e076b39988190933fb8c3fef52201 completed BatchRequestCounts(completed=3, failed=0, total=3)
batch_696e027025108190b3016edb479beca0 completed BatchRequest

In [94]:
import os, time
from openai import OpenAI

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# === 1) 你决定要保留哪个 still running 的 batch ===
# 建议填你当前真正想跑的那个（例如 total=2000 的那个）
KEEP_ID = "batch_696e47e2b80c81908c4402f0561f44be"  # ←←← 改成你要保留的

TERMINAL = {"completed", "failed", "expired", "cancelled"}
RUNNING  = {"validating", "in_progress", "finalizing"}

def download_file(file_id, path):
    if not file_id:
        return False
    resp = client.files.content(file_id)   # binary stream
    data = resp.read()
    with open(path, "wb") as f:
        f.write(data)
    return True

# === 2) 列出最近 batch ===
bs = client.batches.list(limit=50)
print("---- Recent batches ----")
for b in bs.data:
    rc = getattr(b, "request_counts", None)
    print(b.id, b.status, rc)

# === 3) 取消所有 RUNNING 但不是 KEEP_ID 的 batch ===
print("\n---- Cancel extra running batches ----")
for b in bs.data:
    if b.status in RUNNING and b.id != KEEP_ID:
        try:
            client.batches.cancel(b.id)
            print("cancelled:", b.id)
        except Exception as e:
            print("cancel failed:", b.id, e)

# === 4) 下载所有 completed 的 output（以及 error，如果有） ===
out_dir = "/content/batch_outputs"
os.makedirs(out_dir, exist_ok=True)

print("\n---- Download completed outputs ----")
for b in bs.data:
    if b.status == "completed":
        br = client.batches.retrieve(b.id)
        out_id = getattr(br, "output_file_id", None)
        err_id = getattr(br, "error_file_id", None)

        if out_id:
            out_path = f"{out_dir}/{b.id}_output.jsonl"
            ok = download_file(out_id, out_path)
            print("saved output:", out_path, "OK" if ok else "FAIL")
        if err_id:
            err_path = f"{out_dir}/{b.id}_error.jsonl"
            ok = download_file(err_id, err_path)
            print("saved error :", err_path, "OK" if ok else "FAIL")

print("\nAll done. Files are in:", out_dir)


---- Recent batches ----
batch_696e47e2b80c81908c4402f0561f44be in_progress BatchRequestCounts(completed=0, failed=0, total=2000)
batch_696e46b37e148190bc31bf217a6f3935 in_progress BatchRequestCounts(completed=0, failed=0, total=500)
batch_696e42e5a3cc8190af90f8d7487cecdf cancelled BatchRequestCounts(completed=0, failed=0, total=2000)
batch_696e41df8b988190a38d3bb59136699f in_progress BatchRequestCounts(completed=0, failed=0, total=2000)
batch_696e398278288190829dda01fbfc2f3f in_progress BatchRequestCounts(completed=0, failed=0, total=2000)
batch_696e34454dac81908cc588b7cd997232 in_progress BatchRequestCounts(completed=0, failed=0, total=2000)
batch_696e0f3ec1708190b87930cbadffae5d completed BatchRequestCounts(completed=2000, failed=0, total=2000)
batch_696e07e871048190a9dc660ab755441c completed BatchRequestCounts(completed=2000, failed=0, total=2000)
batch_696e076b39988190933fb8c3fef52201 completed BatchRequestCounts(completed=3, failed=0, total=3)
batch_696e027025108190b3016edb479bec

In [96]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/swissmetro_batches
!cp -r /content/batch_outputs/* /content/drive/MyDrive/swissmetro_batches/

!ls -lh /content/drive/MyDrive/swissmetro_batches | head
print("Saved to Drive: /content/drive/MyDrive/swissmetro_batches/")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
total 53M
-rw------- 1 root root 703K Jan 19 15:59 batch_696cdafd8b388190b2e416c4c91a07e6_error.jsonl
-rw------- 1 root root 703K Jan 19 15:59 batch_696ce2e915d08190bbdd98cc629637aa_error.jsonl
-rw------- 1 root root 3.2M Jan 19 15:59 batch_696cee491ee081909f8757b694625c53_output.jsonl
-rw------- 1 root root 3.3K Jan 19 15:59 batch_696cefa4b5008190a401ca953139cf5a_output.jsonl
-rw------- 1 root root 3.5M Jan 19 15:59 batch_696cfdb9a5308190a32a37a46f4ba06d_output.jsonl
-rw------- 1 root root 3.5M Jan 19 15:59 batch_696d0a899ca08190a3026751ac7d49a6_output.jsonl
-rw------- 1 root root 3.5M Jan 19 15:59 batch_696d0fb9252c8190854f28a6ccaf459e_output.jsonl
-rw------- 1 root root 3.5M Jan 19 15:59 batch_696d140912448190922d260b73d15cfc_output.jsonl
-rw------- 1 root root 3.5M Jan 19 15:59 batch_696d186d28688190a83d7fd1d50148e5_output.jsonl
Saved to Drive: /content/d

In [109]:
batch1_id = submit_batch(jsonl1)
out1, err1 = wait_and_download_safe(batch1_id, "/content/batch_stage1_out.jsonl", poll_s=20)


[00:00:00] batch_696e64754a8081909293c4747d53f327 -> validating done 0/0 | failed 0
[00:00:20] batch_696e64754a8081909293c4747d53f327 -> in_progress done 0/2000 | failed 0
[00:00:41] batch_696e64754a8081909293c4747d53f327 -> in_progress done 0/2000 | failed 0
[00:01:01] batch_696e64754a8081909293c4747d53f327 -> in_progress done 0/2000 | failed 0
[00:01:21] batch_696e64754a8081909293c4747d53f327 -> in_progress done 0/2000 | failed 0
[00:01:42] batch_696e64754a8081909293c4747d53f327 -> in_progress done 0/2000 | failed 0
[00:02:02] batch_696e64754a8081909293c4747d53f327 -> in_progress done 0/2000 | failed 0
[00:02:22] batch_696e64754a8081909293c4747d53f327 -> in_progress done 0/2000 | failed 0
[00:02:42] batch_696e64754a8081909293c4747d53f327 -> in_progress done 0/2000 | failed 0
[00:03:03] batch_696e64754a8081909293c4747d53f327 -> in_progress done 0/2000 | failed 0
[00:03:23] batch_696e64754a8081909293c4747d53f327 -> in_progress done 0/2000 | failed 0
[00:03:43] batch_696e64754a808190929

KeyboardInterrupt: 

In [ ]:
def extract_output_text(resp_body):
    # 兼容不同结构：优先 output_text
    if isinstance(resp_body, dict) and resp_body.get("output_text") is not None:
        return resp_body["output_text"]
    if isinstance(resp_body, dict):
        for item in resp_body.get("output", []):
            for c in item.get("content", []):
                if c.get("type") in ["output_text", "text"] and "text" in c:
                    return c["text"]
    return None

def parse_batch_output(out_jsonl_path, N):
    choices = [None] * N
    with open(out_jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            cid = obj.get("custom_id", "")
            # custom_id: req_0, req_1, ...
            if not cid.startswith("req_"):
                continue
            i = int(cid.replace("req_", ""))

            # 失败行可能没有 response/body
            if obj.get("error") is not None:
                continue

            resp = obj.get("response", {})
            body = resp.get("body", {})
            txt = extract_output_text(body)
            if not txt:
                continue

            try:
                d = json.loads(txt)
                choices[i] = int(d["CHOICE"])
            except Exception:
                pass
    return choices

choices1 = parse_batch_output(out1, N)
print("parsed choices:", sum([c is not None for c in choices1]), "/", N)


In [ ]:
syn1 = templates.copy()
syn1["CHOICE"] = choices1

syn1_ok = syn1.dropna(subset=["CHOICE"]).copy()
syn1_ok["CHOICE"] = syn1_ok["CHOICE"].astype(int)

scored1 = score_with_baseline_mnl(syn1_ok, beta, scales)

def chosen_prob(df):
    ch = df["CHOICE"].to_numpy()
    p = np.where(ch==1, df["P_TRAIN"].to_numpy(),
        np.where(ch==2, df["P_SM"].to_numpy(), df["P_CAR"].to_numpy()))
    return p

scored1["P_CHOSEN"] = chosen_prob(scored1)

bad_idx = set(syn1.index[syn1["CHOICE"].isna()].tolist())
bad_idx |= set(scored1.index[scored1["P_CHOSEN"] < 0.01].tolist())

print("stage1 ok:", syn1_ok.shape[0], "/", N)
print("stage1 bad:", len(bad_idx))


In [ ]:
MODEL_STAGE2 = "gpt-4o"
bad_list = sorted(list(bad_idx))
M = len(bad_list)
print("stage2 to_fix:", M)

jsonl2 = "/content/batch_stage2.jsonl"

fmt2 = {
    "type": "json_schema",
    "name": "choice_schema",
    "strict": True,
    "schema": schema
}

with open(jsonl2, "w", encoding="utf-8") as f:
    for i in bad_list:
        row = templates.loc[i]
        body = {
            "model": MODEL_STAGE2,
            "input": [
                {"role": "system", "content": "Return only strict JSON following the schema."},
                {"role": "user", "content": make_prompt(row, cot=True)}
            ],
            "text": {"format": fmt2}
        }
        req = {"custom_id": f"bad_{i}", "method": "POST", "url": "/v1/responses", "body": body}
        f.write(json.dumps(req) + "\n")

print("wrote:", jsonl2)


In [ ]:
if M > 0:
    batch2_id = submit_batch(jsonl2)
    out2, err2 = wait_and_download_safe(batch2_id, "/content/batch_stage2_out.jsonl", poll_s=20)

    # 解析 bad_{i}
    fixes = {}
    with open(out2, "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            cid = obj.get("custom_id", "")
            if not cid.startswith("bad_"):
                continue
            i = int(cid.replace("bad_", ""))
            if obj.get("error") is not None:
                continue
            body = obj.get("response", {}).get("body", {})
            txt = extract_output_text(body)
            if not txt:
                continue
            try:
                d = json.loads(txt)
                fixes[i] = int(d["CHOICE"])
            except Exception:
                pass

    syn_final = syn1.copy()
    for i, ch in fixes.items():
        syn_final.loc[i, "CHOICE"] = ch
else:
    syn_final = syn1.copy()

syn_final = syn_final.dropna(subset=["CHOICE"]).copy()
syn_final["CHOICE"] = syn_final["CHOICE"].astype(int)

print("syn_final N:", len(syn_final))
print("choice share:", syn_final["CHOICE"].value_counts(normalize=True).sort_index())


In [ ]:
row_llm = evaluate_one(syn_final, beta, scales, real_train, real_test, label="LLM_two_stage_batch")
pd.DataFrame([row_llm]).T


In [ ]:
syn_final.to_csv("/content/synthetic_llm_two_stage.csv", index=False)
print("saved:", "/content/synthetic_llm_two_stage.csv")


In [ ]:
def share(s):
    return s.value_counts(normalize=True).reindex([1,2,3], fill_value=0.0)

real_train_share = share(real_train["CHOICE"])
real_test_share  = share(real_test["CHOICE"])
syn_share        = share(syn_final["CHOICE"])

print("real_train:", real_train_share.to_dict())
print("real_test :", real_test_share.to_dict())
print("syn_final :", syn_share.to_dict())

tvd_train = 0.5 * (real_train_share - syn_share).abs().sum()
tvd_test  = 0.5 * (real_test_share  - syn_share).abs().sum()
print("TVD vs train:", tvd_train)
print("TVD vs test :", tvd_test)


In [ ]:
schema_u = {
  "type": "object",
  "properties": {
    "U_TRAIN": {"type": "number"},
    "U_SM":    {"type": "number"},
    "U_CAR":   {"type": "number"}
  },
  "required": ["U_TRAIN","U_SM","U_CAR"],
  "additionalProperties": False
}

fmt_u = {
  "type": "json_schema",
  "name": "utility_schema",
  "strict": True,
  "schema": schema_u
}


In [ ]:
def make_prompt_u(row, cot=False):
    extra = ""
    if cot:
        extra = "\nReason internally; DO NOT output reasoning."
    return f"""You are simulating a discrete choice in Swissmetro.
Output ONLY a strict JSON object matching the schema with keys U_TRAIN, U_SM, U_CAR (real numbers).
Higher utility means more preferred.{extra}

Alternatives:
TRAIN, SWISSMETRO, CAR

Traveler attributes (coded):
MALE={int(row.MALE)}, AGE={int(row.AGE)}, INCOME={int(row.INCOME)}, PURPOSE={int(row.PURPOSE)}, GA={int(row.GA)}.

Level-of-service:
TRAIN: TT={float(row.TRAIN_TT)}, CO={float(row.TRAIN_CO)}, HE={float(row.TRAIN_HE)}
SM:    TT={float(row.SM_TT)},    CO={float(row.SM_CO)},    HE={float(row.SM_HE)}, SEATS={float(row.SM_SEATS)}
CAR:   TT={float(row.CAR_TT)},   CO={float(row.CAR_CO)}

Instruction:
- Do NOT add any special preference for GA beyond its cost implication (no GA preference term).
- Use time/cost/headway/comfort to form utilities.
"""


In [ ]:
MODEL_STAGE1 = "gpt-4o-mini"
jsonl1 = "/content/batch_stage1_util.jsonl"

with open(jsonl1, "w", encoding="utf-8") as f:
    for i in range(N):
        row = templates.loc[i]
        body = {
            "model": MODEL_STAGE1,
            "input": [
                {"role":"system","content":"Return only strict JSON following the schema."},
                {"role":"user","content": make_prompt_u(row, cot=False)}
            ],
            "text": {"format": fmt_u}
        }
        req = {"custom_id": f"req_{i}", "method":"POST", "url":"/v1/responses", "body": body}
        f.write(json.dumps(req) + "\n")
print("wrote:", jsonl1)


In [ ]:
def parse_batch_output_util(out_jsonl_path, N):
    U = np.full((N,3), np.nan, dtype=float)  # columns: train, sm, car
    with open(out_jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            cid = obj.get("custom_id","")
            if not cid.startswith("req_"):
                continue
            i = int(cid.replace("req_",""))
            if obj.get("error") is not None:
                continue
            body = obj.get("response", {}).get("body", {})
            txt = extract_output_text(body)
            if not txt:
                continue
            try:
                d = json.loads(txt)
                U[i,0] = float(d["U_TRAIN"])
                U[i,1] = float(d["U_SM"])
                U[i,2] = float(d["U_CAR"])
            except:
                pass
    return U

def softmax(x, tau=1.0):
    x = x / tau
    x = x - np.nanmax(x, axis=1, keepdims=True)
    ex = np.exp(x)
    ex_sum = np.nansum(ex, axis=1, keepdims=True)
    return ex / ex_sum

# submit + download
batch1_id = submit_batch(jsonl1)
out1, err1 = wait_and_download_safe(batch1_id, "/content/batch_stage1_util_out.jsonl", poll_s=20)

U = parse_batch_output_util(out1, N)
ok = np.isfinite(U).all(axis=1)
print("parsed utilities:", ok.sum(), "/", N)

tau = 1.2  # 你可以试 0.8~2.0，tau越大越分散
P = softmax(U[ok], tau=tau)

rng = np.random.default_rng(123)
choices = []
for p in P:
    # 1=train,2=sm,3=car
    choices.append(1 + rng.choice(3, p=p))

syn_u = templates.loc[np.where(ok)[0]].copy()
syn_u["CHOICE"] = choices


In [ ]:
print("syn choice share:", syn_u["CHOICE"].value_counts(normalize=True).sort_index())

row_llm_u = evaluate_one(syn_u, beta, scales, real_train, real_test, label=f"LLM_util_softmax_tau{tau}")
pd.DataFrame([row_llm_u]).T


In [ ]:
import json, numpy as np, pandas as pd

def parse_batch_output_util(out_jsonl_path, N):
    """Parse batch output jsonl -> utilities matrix U (N,3): train, sm, car"""
    U = np.full((N,3), np.nan, dtype=float)
    with open(out_jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            cid = obj.get("custom_id","")
            if not cid.startswith("req_"):
                continue
            i = int(cid.replace("req_",""))
            # batch line has either {"response":{...}} or {"error":...}
            if obj.get("error") is not None:
                continue
            body = obj.get("response", {}).get("body", {})
            txt = extract_output_text(body)   # 你 notebook 里已经有这个函数
            if not txt:
                continue
            try:
                d = json.loads(txt)
                U[i,0] = float(d["U_TRAIN"])
                U[i,1] = float(d["U_SM"])
                U[i,2] = float(d["U_CAR"])
            except:
                pass
    return U

def softmax_rows(U, tau=1.0):
    """row-wise softmax with temperature tau"""
    X = U / tau
    X = X - np.max(X, axis=1, keepdims=True)
    ex = np.exp(X)
    return ex / ex.sum(axis=1, keepdims=True)

def generate_from_utilities_batch(
    templates,
    model="gpt-4o-mini",
    tau=1.0,
    jsonl_path="/content/tmp_util_batch.jsonl",
    out_path="/content/tmp_util_out.jsonl",
    seed=123,
    cot=False,
):
    """
    1) Write batch jsonl for utilities schema
    2) Submit batch -> download out jsonl
    3) Parse utilities -> softmax(tau) -> sample CHOICE
    Return syn_u df (same rows as templates) with CHOICE column
    """
    N = len(templates)

    # ---- schema + fmt (utilities) ----
    schema_u = {
      "type": "object",
      "properties": {
        "U_TRAIN": {"type": "number"},
        "U_SM":    {"type": "number"},
        "U_CAR":   {"type": "number"}
      },
      "required": ["U_TRAIN","U_SM","U_CAR"],
      "additionalProperties": False
    }
    fmt_u = {
      "type": "json_schema",
      "name": "utility_schema",
      "strict": True,
      "schema": schema_u
    }

    # ---- write jsonl ----
    with open(jsonl_path, "w", encoding="utf-8") as f:
        for i in range(N):
            row = templates.loc[i]
            body = {
                "model": model,
                "input": [
                    {"role":"system","content":"Return only strict JSON following the schema."},
                    {"role":"user","content": make_prompt_u(row, cot=cot)}  # 你已经定义的 make_prompt_u
                ],
                "text": {"format": fmt_u}
            }
            req = {"custom_id": f"req_{i}", "method":"POST", "url":"/v1/responses", "body": body}
            f.write(json.dumps(req) + "\n")

    # ---- submit + download ----
    batch_id = submit_batch(jsonl_path)  # 你 notebook 里已有
    out_jsonl, err_jsonl = wait_and_download_safe(batch_id, out_path, poll_s=20)  # 你 notebook 里已有

    # ---- parse utilities ----
    U = parse_batch_output_util(out_jsonl, N)
    ok = np.isfinite(U).all(axis=1)
    print(f"parsed utilities: {ok.sum()} / {N}")
    if ok.sum() < N:
        # 保险：如果有少量解析失败，先丢弃（你也可以用 stage2 repair）
        templates_ok = templates.loc[np.where(ok)[0]].copy().reset_index(drop=True)
        U_ok = U[ok]
    else:
        templates_ok = templates.copy().reset_index(drop=True)
        U_ok = U

    # ---- sample CHOICE from softmax ----
    P = softmax_rows(U_ok, tau=tau)
    rng = np.random.default_rng(seed)
    choice = 1 + np.array([rng.choice(3, p=p) for p in P])  # 1=train,2=sm,3=car

    syn_u = templates_ok.copy()
    syn_u["CHOICE"] = choice.astype(int)
    return syn_u


In [ ]:
taus = [0.2, 0.5, 0.8, 1.0, 1.2]   # 先别太多
rows = []

for tau in taus:
    syn_u = generate_from_utilities_batch(
        templates,
        model="gpt-4o-mini",
        tau=tau,
        jsonl_path=f"/content/util_tau{tau}.jsonl",
        out_path=f"/content/util_tau{tau}_out.jsonl",
        seed=123,
        cot=False
    )

    # 真实 choice share（不是 pred share）
    share = syn_u["CHOICE"].value_counts(normalize=True).reindex([1,2,3]).fillna(0.0)

    row = evaluate_one(syn_u, beta, scales, real_train, real_test, label=f"util_softmax_tau{tau}")
    row["choice_share_train"] = float(share.loc[1])
    row["choice_share_sm"]    = float(share.loc[2])
    row["choice_share_car"]   = float(share.loc[3])

    rows.append(row)

df_tau = pd.DataFrame(rows).set_index("label")
df_tau


In [ ]:
# === 1) 跑一次 baseline scoring，拿到输出df ===
tmp = score_with_baseline_mnl(real_train.head(50).copy(), beta, scales)

print("score_with_baseline_mnl 输出列数:", len(tmp.columns))
print("前30个列名:", list(tmp.columns)[:30])
print("最后30个列名:", list(tmp.columns)[-30:])

# === 2) 重点检查：有没有 deterministic utilities / logits 之类的列 ===
cands = [c for c in tmp.columns if any(k in c.upper() for k in [
    "V_", "UTIL", "U_", "LOGIT", "SCORE"
])]
print("\n可能是效用/对数几率的列名:", cands)

# === 3) 也顺便检查概率列是否都在 ===
prob_cols = [c for c in tmp.columns if c.upper().startswith("P_")]
print("\n概率相关列名:", prob_cols)

# === 4) 如果没有 V_*，至少确认我们能从函数里拿到 P_*（后面也能反推logit差） ===
print("\nP_TRAIN/P_SM/P_CAR 是否存在:",
      all(x in tmp.columns for x in ["P_TRAIN","P_SM","P_CAR"]))


In [ ]:
import json, numpy as np, pandas as pd

def parse_batch_output_residual(out_jsonl_path, N):
    """Parse batch output jsonl -> residual matrix dU (N,3)"""
    dU = np.full((N,3), np.nan, dtype=float)
    with open(out_jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            cid = obj.get("custom_id","")
            if not cid.startswith("req_"):
                continue
            i = int(cid.replace("req_",""))
            if obj.get("error") is not None:
                continue
            body = obj.get("response", {}).get("body", {})
            txt = extract_output_text(body)   # 你已有
            if not txt:
                continue
            try:
                r = json.loads(txt)
                dU[i,0] = float(r["dU_TRAIN"])
                dU[i,1] = float(r["dU_SM"])
                dU[i,2] = float(r["dU_CAR"])
            except:
                pass
    return dU

def softmax_rows(X):
    X = X - np.max(X, axis=1, keepdims=True)
    ex = np.exp(X)
    return ex / ex.sum(axis=1, keepdims=True)

def make_prompt_residual(row, cot=False):
    # 这里不需要 LLM 计算效用，只要输出小扰动（偏好残差）
    extra = ""
    if cot:
        extra = "\n(Think internally: compare time and cost; output JSON only.)"

    return f"""You will provide a small preference residual for Swissmetro choice.
Output ONLY strict JSON matching the schema. Each residual must be within [-1, 1].

Alternatives:
1=TRAIN, 2=SWISSMETRO, 3=CAR

Traveler attributes (coded):
MALE={int(row.MALE)}, AGE={int(row.AGE)}, INCOME={int(row.INCOME)}, PURPOSE={int(row.PURPOSE)}, GA={int(row.GA)}.

Level-of-service:
TRAIN: TT={float(row.TRAIN_TT)}, CO={float(row.TRAIN_CO)}, HE={float(row.TRAIN_HE)}
SM:    TT={float(row.SM_TT)},    CO={float(row.SM_CO)},    HE={float(row.SM_HE)}, SEATS={float(row.SM_SEATS)}
CAR:   TT={float(row.CAR_TT)},   CO={float(row.CAR_CO)}

Task:
- Return residuals that slightly adjust preferences but do NOT override realistic trade-offs.
- Do NOT add any special GA preference term.
- Keep values small. {extra}
"""

def generate_from_residual_batch(
    templates,
    model="gpt-4o-mini",
    lam=0.3,          # λ：LLM扰动强度（建议0.1~0.5扫）
    jsonl_path="/content/batch_resid.jsonl",
    out_path="/content/batch_resid_out.jsonl",
    seed=123,
    cot=False
):
    """
    1) baseline logits = log(P_j) from baseline MNL
    2) LLM outputs dU in [-1,1]
    3) logits' = log(P) + lam*dU
    4) softmax sample -> CHOICE
    """
    N = len(templates)

    # 先算 baseline P（你已有的函数）
    scored = score_with_baseline_mnl(templates.copy(), beta, scales)
    P = scored[["P_TRAIN","P_SM","P_CAR"]].to_numpy()

    # 防止 log(0)
    eps = 1e-12
    base_logits = np.log(np.clip(P, eps, 1.0))

    # residual schema
    schema_r = {
      "type": "object",
      "properties": {
        "dU_TRAIN": {"type": "number", "minimum": -1.0, "maximum": 1.0},
        "dU_SM":    {"type": "number", "minimum": -1.0, "maximum": 1.0},
        "dU_CAR":   {"type": "number", "minimum": -1.0, "maximum": 1.0}
      },
      "required": ["dU_TRAIN","dU_SM","dU_CAR"],
      "additionalProperties": False
    }
    fmt_r = {"type":"json_schema","name":"residual_schema","strict":True,"schema":schema_r}

    # write jsonl
    with open(jsonl_path, "w", encoding="utf-8") as f:
        for i in range(N):
            row = templates.loc[i]
            body = {
                "model": model,
                "input": [
                    {"role":"system","content":"Return only strict JSON following the schema."},
                    {"role":"user","content": make_prompt_residual(row, cot=cot)}
                ],
                "text": {"format": fmt_r}
            }
            req = {"custom_id": f"req_{i}", "method":"POST", "url":"/v1/responses", "body": body}
            f.write(json.dumps(req) + "\n")

    # submit + download
    batch_id = submit_batch(jsonl_path)
    out_jsonl, err_jsonl = wait_and_download_safe(batch_id, out_path, poll_s=20)

    dU = parse_batch_output_residual(out_jsonl, N)
    ok = np.isfinite(dU).all(axis=1)
    print(f"parsed residuals: {ok.sum()} / {N}")

    # 对失败行：先用0扰动（最稳），不丢数据
    dU[~ok] = 0.0

    logits = base_logits + lam * dU
    P_new = softmax_rows(logits)

    rng = np.random.default_rng(seed)
    choice = 1 + np.array([rng.choice(3, p=p) for p in P_new])  # 1/2/3

    syn = templates.copy()
    syn["CHOICE"] = choice.astype(int)
    return syn


In [ ]:
lams = [0.0, 0.1, 0.2, 0.3, 0.5]
rows = []

for lam in lams:
    syn_r = generate_from_residual_batch(
        templates,
        model="gpt-4o-mini",
        lam=lam,
        jsonl_path=f"/content/batch_resid_lam{lam}.jsonl",
        out_path=f"/content/batch_resid_lam{lam}_out.jsonl",
        seed=123,
        cot=False
    )

    share = syn_r["CHOICE"].value_counts(normalize=True).reindex([1,2,3]).fillna(0.0)
    row = evaluate_one(syn_r, beta, scales, real_train, real_test, label=f"mnl_plus_resid_lam{lam}")
    row["choice_share_train"] = float(share.loc[1])
    row["choice_share_sm"]    = float(share.loc[2])
    row["choice_share_car"]   = float(share.loc[3])
    rows.append(row)

df_lam = pd.DataFrame(rows).set_index("label")
df_lam


In [ ]:
import re, types, inspect, sys
import pandas as pd
import numpy as np

# ---- 1) 在当前全局命名空间里找候选函数 ----
pat = re.compile(r"(estimate|fit|train|mnl|logit|biogeme|pylogit|choice|mlogit|maxlik)", re.I)

cands = []
for name, obj in list(globals().items()): # Iterate over a copy
    if isinstance(obj, types.FunctionType) and pat.search(name):
        try:
            sig = str(inspect.signature(obj))
        except Exception:
            sig = "(signature unavailable)"
        cands.append((name, sig, obj.__module__))

cands = sorted(cands, key=lambda x: x[0].lower())

print(f"Found {len(cands)} candidate functions in globals():\n")
for i,(name,sig,mod) in enumerate(cands[:200]):  # 最多先显示200个
    print(f"[{i:03d}] {name}{sig}   (module={mod})")

# ---- 2) 如果你用的是某个类/对象方法（比如 model.fit），也帮你列出常见对象 ----
print("\n---- Objects that might hold estimators (models) ----")
obj_pat = re.compile(r"(model|mnl|logit|biogeme|estimator|trainer|choice)", re.I)
objs = []
for name, obj in list(globals().items()): # Iterate over a copy
    if obj_pat.search(name) and not isinstance(obj, types.FunctionType):
        objs.append((name, type(obj)))
for name, typ in sorted(objs, key=lambda x: x[0].lower())[:60]:
    print(f"{name}: {typ}")

In [ ]:
import numpy as np
import pandas as pd
import os

# ============ 0) 小工具：从 scales 或全局变量里拿缩放 ============
def _get_scale(scales, key, fallback_global):
    if isinstance(scales, dict) and key in scales:
        return scales[key]
    return globals().get(fallback_global, None)

tt_scale = _get_scale(globals().get("scales", None), "tt_scale", "tt_scale")
co_scale = _get_scale(globals().get("scales", None), "co_scale", "co_scale")
he_scale = _get_scale(globals().get("scales", None), "he_scale", "he_scale")

if tt_scale is None or co_scale is None or he_scale is None:
    raise ValueError("没找到 tt_scale/co_scale/he_scale。请确认你前面已经计算并存在 scales 或同名全局变量里。")

# ============ 1) 选择你要用的“估计器” ============
# 优先用你列表里那个 fit_mnl_step2_main；没有就用 fit_step2_main
if "fit_mnl_step2_main" in globals():
    _EST = globals()["fit_mnl_step2_main"]
elif "fit_step2_main" in globals():
    _EST = globals()["fit_step2_main"]
else:
    raise NameError("找不到 fit_mnl_step2_main / fit_step2_main。请确认这两个函数在 notebook 里已定义。")

# ============ 2) 估计函数：输入 df -> 输出 theta（向量） ============
REQ_COLS = [
    "CHOICE",
    "TRAIN_TT","TRAIN_CO","TRAIN_HE",
    "SM_TT","SM_CO","SM_HE","SM_SEATS",
    "CAR_TT","CAR_CO",
    "FIRST","MALE","AGE","INCOME","PURPOSE"
]

def estimate_theta_step2main(df, theta_init=None, verbose=True):
    d = df.copy()

    # 强制数值化（避免 object/string）
    for c in REQ_COLS:
        if c in d.columns:
            d[c] = pd.to_numeric(d[c], errors="coerce")

    d = d.dropna(subset=[c for c in REQ_COLS if c in d.columns]).copy()

    # 你这套 step2(main) 的估计器签名就是 (df_sub, tt_scale, co_scale, he_scale, theta_init=...)
    res = _EST(d, tt_scale, co_scale, he_scale, theta_init=theta_init)

    if verbose:
        print("estimator:", _EST.__name__)
        print("N used:", len(d))
        print("success:", getattr(res, "success", None), "fun:", getattr(res, "fun", None))

    theta = res.x if hasattr(res, "x") else res  # 兼容返回 x 或直接返回 theta
    return theta, res

# ============ 3) 把 theta 转成和 Appendix_A1_full_params.csv 一样的 param 命名 ============
# 依赖你 notebook 里已有：unpack_theta2(theta), get_X_names()
if "unpack_theta2" not in globals() or "get_X_names" not in globals():
    raise NameError("缺少 unpack_theta2 / get_X_names。请确认这两个函数在 notebook 里已定义。")

def theta_to_param_series(theta):
    # 确保 K_main 存在，它是 main 模型的 X_ind 维度
    if "K_main" not in globals():
        raise NameError("K_main 未定义。请确保你运行了计算 K_main 的相关单元格。")

    # 1) 先解包成3部分
    head, gamma_sm, gamma_car = unpack_theta2(theta, K_main)

    # 2) 再把 head 解包成 6 个基础参数
    b_tt, b_co, b_he, b_seats, asc_sm, asc_car = head

    X_names_all = get_X_names()
    X = [n for n in X_names_all if n != "FIRST"] # main model 不含 FIRST

    out = {
        "B_TT": float(b_tt),
        "B_CO": float(b_co),
        "B_HE": float(b_he),
        "B_SEATS": float(b_seats),
        "ASC_SM": float(asc_sm),
        "ASC_CAR": float(asc_car),
    }
    for name, val in zip(X, gamma_sm):
        out[f"G_SM:{name}"] = float(val)
    for name, val in zip(X, gamma_car):
        out[f"G_CAR:{name}"] = float(val)

    return pd.Series(out)

# ============ 4) 做“参数稳定性”：baseline vs augmented ============
def make_augmented(real_train, syn_df, ratio=1.0, seed=0):
    """
    ratio=1.0 表示抽取 len(real_train) 行 synthetic（不够就放回采样），然后拼接在 real_train 后面。
    你也可以 ratio=0.5 / 2.0 等。
    """
    n = int(len(real_train) * ratio)
    replace = len(syn_df) < n
    syn_samp = syn_df.sample(n=n, replace=replace, random_state=seed)
    return pd.concat([real_train, syn_samp], ignore_index=True)

def param_stability_table(theta_base, theta_new, appendix_csv_path=None):
    s_base = theta_to_param_series(theta_base).rename("coef_base")
    s_new  = theta_to_param_series(theta_new).rename("coef_new")
    out = pd.concat([s_base, s_new], axis=1)

    out["diff"] = out["coef_new"] - out["coef_base"]
    out["abs_diff"] = out["diff"].abs()

    # 如果你提供 Appendix_A1_full_params.csv，就用 boot_se 做 z_diff（更直观）
    if appendix_csv_path is not None and os.path.exists(appendix_csv_path):
        apx = pd.read_csv(appendix_csv_path).set_index("param")
        out = out.join(apx[["boot_se","ci_2.5","ci_97.5","p_boot","coef"]].rename(columns={"coef":"coef_reported"}), how="left")
        out["z_diff_vs_bootse"] = out["diff"] / out["boot_se"].replace(0, np.nan)

    return out.sort_values("abs_diff", ascending=False)

# ============ 5) 你实际怎么跑 ============
# (A) baseline：在 real_train 上再估一次（保证一致）
theta_base, res_base = estimate_theta_step2main(real_train, theta_init=None, verbose=True)

# (B) augmented：用你某个 synthetic_df（例如 syn_final / syn_u / 你刚生成的csv读进来）
# 如果你现在想用变量 syn_final：
# df_aug = make_augmented(real_train, syn_final, ratio=1.0, seed=0)

# 如果你是从文件读：
# syn_final = pd.read_csv("/content/synthetic_llm_two_stage.csv")
# df_aug = make_augmented(real_train, syn_final, ratio=1.0, seed=0)

# 这里先写成你自己替换 syn_df 变量名：
syn_df = syn_final  # <<< 改成你要评估的那个 synthetic dataframe 变量名
df_aug = make_augmented(real_train, syn_df, ratio=1.0, seed=0)

theta_aug, res_aug = estimate_theta_step2main(df_aug, theta_init=theta_base, verbose=True)

# (C) 输出稳定性表
apx_path = "/content/Appendix_A1_full_params.csv"  # 你也可以改成自己实际路径
stab = param_stability_table(theta_base, theta_aug, appendix_csv_path=apx_path)

display(stab.head(25))

# (D) 存盘
out_path = "/content/param_stability_real_vs_aug.csv"
stab.to_csv(out_path, index=True)
print("saved:", out_path)

In [100]:
import numpy as np
import pandas as pd

def build_step2_data(df_sub, tt_scale, co_scale, he_scale):
    # y: 1=Train, 2=SM, 3=Car  -> 转成 0/1/2
    y = df_sub["CHOICE"].values.astype(int) - 1

    AV = np.column_stack([
        df_sub["TRAIN_AV"].values,
        df_sub["SM_AV"].values,
        df_sub["CAR_AV"].values
    ]).astype(float)

    TT = np.column_stack([
        df_sub["TRAIN_TT"].values,
        df_sub["SM_TT"].values,
        df_sub["CAR_TT"].values
    ]).astype(float) / tt_scale

    CO = np.column_stack([
        df_sub["TRAIN_CO"].values,
        df_sub["SM_CO"].values,
        df_sub["CAR_CO"].values
    ]).astype(float) / co_scale

    HE = np.column_stack([
        df_sub["TRAIN_HE"].values,
        df_sub["SM_HE"].values,
        df_sub["CAR_HE"].values
    ]).astype(float) / he_scale

    SEATS = np.column_stack([
        np.zeros(len(df_sub)),
        df_sub["SM_SEATS"].values.astype(float),
        np.zeros(len(df_sub))
    ])

    data = {"y": y, "AV": AV, "TTs": TT, "COs": CO, "HEs": HE, "SEATS": SEATS}
    X_ind = get_X_ind(df_sub)  # 你 notebook 已经有
    return data, X_ind


In [101]:
def estimate_step2_main_reportable(df_sub, tt_scale, co_scale, he_scale,
                                  theta_init=None,
                                  rounds=5, maxfun=80000, maxiter=80000,
                                  ftol=1e-7, gtol=1e-6):
    # 先跑一次（给一个合理初值）
    res0 = fit_step2_main(df_sub, tt_scale, co_scale, he_scale, theta_init=theta_init,
                         maxfun=20000, maxiter=20000)

    data, X_ind = build_step2_data(df_sub, tt_scale, co_scale, he_scale)
    negll = make_negll_step2(data, X_ind)

    # 续跑（关键修正：把 success 跑出来）
    res = continue_lbfgsb(negll, res0.x, rounds=rounds,
                          maxfun=maxfun, maxiter=maxiter, ftol=ftol, gtol=gtol)

    print("final success:", res.success, "| fun:", res.fun, "| message:", res.message)
    return res.x, res


In [102]:
# 自动确定 K（交互项个数）
X_names = get_X_names()
K_main = len(X_names)

def theta_to_param_series(theta, K=K_main):
    b_tt, b_co, b_he, b_seats, asc_sm, asc_car, gamma_sm, gamma_car = unpack_theta2(theta, K)

    s = {}
    s["B_TT"] = b_tt
    s["B_CO"] = b_co
    s["B_HE"] = b_he
    s["B_SEATS"] = b_seats
    s["ASC_SM"] = asc_sm
    s["ASC_CAR"] = asc_car

    for k, name in enumerate(X_names):
        s[f"G_SM:{name}"]  = gamma_sm[k]
        s[f"G_CAR:{name}"] = gamma_car[k]
    return pd.Series(s)

def param_stability_table_reportable(theta_base, theta_new, appendix_csv_path):
    base = theta_to_param_series(theta_base)
    new  = theta_to_param_series(theta_new)

    out = pd.DataFrame({
        "coef_base": base,
        "coef_new":  new
    })
    out["diff"] = out["coef_new"] - out["coef_base"]
    out["abs_diff"] = out["diff"].abs()

    # 读取你之前 bootstrap 的 Appendix_A1_full_params.csv（里面有 boot_se / ci / p 等）
    apx = pd.read_csv(appendix_csv_path)
    # 兼容列名（你之前表里是 boot_se / ci_2.5 / ci_97.5 / p_boot 之类）
    # 这里假设 apx 至少有 param_name + boot_se
    # 如果你的列名不同，把下面这行的 "param_name" 改成你的实际列名
    apx = apx.rename(columns={"param":"param_name"}) if "param" in apx.columns and "param_name" not in apx.columns else apx

    apx = apx.set_index("param_name")

    # 合并 boot_se（以及你有的话：ci_2.5/ci_97.5/p_boot）
    for c in ["boot_se", "ci_2.5", "ci_97.5", "p_boot"]:
        if c in apx.columns:
            out[c] = apx[c]

    # z_diff_vs_bootse：用 bootstrap se 评估“差异是否超过不确定性”
    if "boot_se" in out.columns:
        se = out["boot_se"].replace(0, np.nan)
        out["z_diff_vs_bootse"] = out["diff"] / se

    return out


In [107]:
import os, glob, pandas as pd

# ---------- helper: find DataFrame in globals ----------
def _pick_df_from_globals(name_list):
    g = globals()
    for n in name_list:
        if n in g and isinstance(g[n], pd.DataFrame):
            return g[n], f"globals()['{n}']"
    return None, None

def _find_df_by_columns(required_cols):
    cands = []
    for n, v in globals().items():
        if isinstance(v, pd.DataFrame):
            if set(required_cols).issubset(set(v.columns)):
                cands.append((n, len(v), v))
    cands.sort(key=lambda x: -x[1])  # largest first
    if cands:
        n, L, df = cands[0]
        return df, f"globals()['{n}'] (auto by columns, N={L})"
    return None, None

def _load_latest_synthetic_csv():
    # 你之前保存过 /content/synthetic_llm_two_stage.csv 等
    patterns = [
        "/content/synthetic*.csv",
        "/content/*synthetic*.csv",
        "/content/drive/MyDrive/**/*.csv",
    ]
    files = []
    for pat in patterns:
        files.extend(glob.glob(pat, recursive=True))
    # 只保留看起来像合成数据的
    files = [f for f in files if os.path.basename(f).lower().startswith(("synthetic", "syn_")) or "synthetic" in os.path.basename(f).lower()]
    if not files:
        return None, None
    files.sort(key=lambda f: os.path.getmtime(f), reverse=True)
    f0 = files[0]
    return pd.read_csv(f0), f0

# ---------- 1) get real_train ----------
REQ_REAL = ["CHOICE","TRAIN_TT","SM_TT","CAR_TT","TRAIN_CO","SM_CO","CAR_CO","TRAIN_HE","SM_HE","CAR_HE",
            "TRAIN_AV","SM_AV","CAR_AV","SM_SEATS"]
real_train, src_real = _pick_df_from_globals(["real_train","df_real_train","train_df","df_train"])
if real_train is None:
    real_train, src_real = _find_df_by_columns(REQ_REAL)

if real_train is None:
    raise RuntimeError(
        "没找到 real_train（也没找到包含核心列的训练集DataFrame）。\n"
        "请先回到你 notebook 里“读取/清洗 Swissmetro + train/test split”的 cell 重新运行，确保 real_train 在内存里。"
    )
print("✅ Using real_train from:", src_real, "| shape:", real_train.shape)

# ---------- 2) get synthetic df (syn_use) ----------
# 优先顺序：你常用的变量名 -> 任何含核心列的df -> 从 /content 里读最新 synthetic*.csv
syn_use, src_syn = _pick_df_from_globals([
    "syn_final","syn_u","syn","syn_df","df_syn","synthetic_df",
    "syn_mnl_plus_resid","mnl_plus_resid_df","syn_resid",
])
if syn_use is None:
    syn_use, src_syn = _find_df_by_columns(REQ_REAL)  # 可能你合成df也在globals里但名字不在列表
    # 避免把 real_train 自己选中
    if syn_use is not None and syn_use is real_train:
        syn_use, src_syn = None, None

if syn_use is None:
    syn_use, src_syn = _load_latest_synthetic_csv()

if syn_use is None:
    raise RuntimeError(
        "没找到合成数据（syn_final/syn_u等变量不存在，/content 下也没找到 synthetic*.csv）。\n"
        "请确认你已经跑过生成合成数据并保存成CSV（比如 synthetic_llm_two_stage.csv）。"
    )
print("✅ Using synthetic df from:", src_syn, "| shape:", syn_use.shape)

# ---------- 3) build augmented df (real_train + syn_use) with aligned columns ----------
common_cols = [c for c in real_train.columns if c in syn_use.columns]
df_aug = pd.concat([real_train[common_cols], syn_use[common_cols]], ignore_index=True)

print("✅ df_aug built. common_cols:", len(common_cols), "| shape:", df_aug.shape)

# quick sanity check
need = set(REQ_REAL)
miss_real = list(need - set(real_train.columns))
miss_syn  = list(need - set(syn_use.columns))
print("Missing in real_train:", miss_real)
print("Missing in syn_use   :", miss_syn)


✅ Using real_train from: globals()['real_train'] | shape: (8568, 28)
✅ Using synthetic df from: globals()['synthetic_df'] | shape: (2000, 28)
✅ df_aug built. common_cols: 28 | shape: (10568, 28)
Missing in real_train: ['CAR_HE']
Missing in syn_use   : ['CAR_HE']


In [108]:
# 例：你之前那段里是 df_aug = real_train + syn_final
# 现在直接用 df_aug（上面已经拼好了）

tt_scale = scales["tt_scale"]
co_scale = scales["co_scale"]
he_scale = scales["he_scale"]

theta_base, res_base = estimate_step2_main_reportable(real_train, tt_scale, co_scale, he_scale)
theta_aug,  res_aug  = estimate_step2_main_reportable(df_aug,     tt_scale, co_scale, he_scale, theta_init=theta_base)

apx_path = "/content/Appendix_A1_full_params.csv"
stab = param_stability_table_reportable(theta_base, theta_aug, apx_path)

stab.to_csv("/content/param_stability_real_vs_aug_reportable.csv", index=True)
show_cols = [c for c in ["coef_base","coef_new","diff","abs_diff","boot_se","p_boot","z_diff_vs_bootse"] if c in stab.columns]
stab_big = stab.sort_values("abs_diff", ascending=False)[show_cols].head(30)
stab_big.to_csv("/content/param_stability_top30_changes.csv", index=True)

print("saved:",
      "/content/param_stability_real_vs_aug_reportable.csv",
      "/content/param_stability_top30_changes.csv")
stab_big


KeyError: 'CAR_HE'